# Lib def

In [1]:
import numpy as np
from sympy.simplify.fu import fu, L, TR9, TR10i, TR11
from sympy import factor, sin, cos, powsimp, exp
from sympy import re, im, I, E, Abs, S, conjugate
from sympy import symbols, Function, lambdify, simplify, cancel, factor, collect, expand, preorder_traversal, Float, latex, pprint
from IPython.display import display
from math import pi

def tensor_product(a, b):
    if ("__len__" not in dir(a)):
        return a*b
    elif ("__len__" not in dir(b)):
        return b*a
    else:
        res = [tensor_product(a[i//len(b)], b[i%len(b)]) for i in range(len(a)*len(b))]
        return np.array(res)

def transform_vect(M, v):
    if (len(M) == len(v) and len(M[0]) == len(v)):
        return matmul(M, v)
    else:
        return v

def matmul(a, b):
    return np.matmul(a, b)

In [2]:
from sympy import init_printing

init_printing(use_latex='mathjax')

In [3]:
def diag_ones(dim):
    return np.array([[int(j == i) for j in range(dim)] for i in range(dim)], dtype='object')

def id(n):
    return diag_ones(2**n)

def rz(theta):
    return np.array([[exp(-I*theta/2), 0], [0, exp(I*theta/2)]])

def rx(theta):
    return np.array([[cos(theta/2), -I*sin(theta/2)], [-I*sin(theta/2), cos(theta/2)]])

def ry(theta):
    return np.array([[cos(theta/2), -sin(theta/2)], [sin(theta/2), cos(theta/2)]])

def p(theta):
    return np.array([[1, 0], [0, exp(I*theta/2)]])
    
def x():
    return np.array([[0, 1], [1, 0]])

def y():
    return np.array([[0, -1j], [1j, 0]])

def z():
    return np.array([[1, 0], [0, -1]])

def cnot(direction=1):
    return cgate(x(), direction)

def cgate(M, direction=1):
    base = diag_ones(2*len(M))

    if direction == 1:
        k = len(M)
    else:
        k = 0

    for i in range(len(M)):
        for j in range(len(M[0])):
            base[i+k][j+k] = M[i][j]

    return np.array(base)

def cgate2(M, c, t, n=None):
    if n is None:
        n = max(c, t)+1

    ket_0 = np.array([1, 0])
    ket_1 = np.array([0, 1])

    proj0x0 = np.outer(ket_0, dagger(ket_0))
    proj1x1 = np.outer(ket_1, dagger(ket_1))

    proj0s = [id(1) for _ in range(n)]
    proj0s[c] = proj0x0
    proj0s[t] = id(1)

    proj1s = [id(1) for _ in range(n)]
    proj1s[c] = proj1x1
    proj1s[t] = M

    return np.array(mtp2(proj0s)+mtp2(proj1s))

def dagger(M):
    return M.transpose()

def conjugate(M):
    res = [[i for i in row] for row in M]

    for i in range(len(M)):
        for j in range(len(M)):
            if type(res[i][j]) == type(1+1j):
                res[i][j] = np.conjugate(res[i][j])
            if type(res[i][j] == e):
                res[i][j] = e(-res[i][j].args())
            elif type(-res[i][j] == e):
                res[i][j] = -e((-res[i][j]).args())

    return res

def idx_to_vect(bit_str):
    ket_0 = np.array([1, 0])
    ket_1 = np.array([0, 1])

    kets = [ket_0 if b == "0" else ket_1 for b in bit_str]

    return multitensor_product(kets)

def braket(sv1, sv2):
    sv1_dagger = dagger(sv1)

    res = matmul(sv1_dagger, sv2)

    return res

def multitensor_product(mats):
    res = None

    for mat in mats:
        if res is None:
            res = mat
        else:
            res = tensor_product(res, mat)

    return np.array(res)

def mtp2(mats):
    res = None
    tempmat = None

    for mat in mats[::-1]:
        if res is None:
            if tempmat is None:
                tempmat = mat
            else:
                res = tensor_product(mat, tempmat)
        else:
            res = tensor_product(mat, res)

    return res

def idx_to_vect2(bit_str):
    ket_0 = np.array([1, 0])
    ket_1 = np.array([0, 1])

    kets = [ket_0 if b == "0" else ket_1 for b in bit_str]

    return mtp2(kets)

def tfv(M, v):
    return transform_vect(M, v)

ket_0 = np.array([1, 0])
ket_1 = np.array([0, 1])
H = 1/np.sqrt(2)*np.array([[1, 1], [1, -1]])

<!-- ![Sample SimpleSA circuit](../data/misc/sample-simplesa-circuit.png) -->
<img src="../data/misc/sample-simplesa-circuit.png" alt="Sample SimpleSA circuit" width=300px align=left>

# Circuit Test

In [37]:
x0 = symbols('x_0', real=True)
x1 = symbols('x_1', real=True)
x2 = symbols('x_2', real=True)
pi = symbols('\\pi', real=True)
# pi = 3.141592653589

In [38]:
psi0 = idx_to_vect("00")
psi1 = tfv(mtp2([rx(pi/2), rx(pi/2)]), psi0)
psi2 = tfv(cgate2(x(), 0, 1), psi1)
psi3 = tfv(mtp2([id(1), p(2*(pi-x0)*(pi-x1))]), psi2)
psi4 = tfv(cgate2(x(), 0, 1), psi3)
psi5 = tfv(mtp2([rx(-pi/2), rx(-pi/2)]), psi4)

psi = psi5

In [39]:
# derive expression for <00|psi>

raw00 = braket(idx_to_vect("00"), psi).expand(trig=True).expand()

braket00 = simplify(raw00, maxn=17, chop=True)
temp = braket00
for a in preorder_traversal(temp):
    if isinstance(a, Float):
        braket00 = braket00.subs(a, round(a, 15))

display(braket00)

⎛                  ⎛   2        ⎞                                           ⎞ 
⎜                ⅈ⋅⎝\pi  + x₀⋅x₁⎠                                           ⎟ 
⎜(1 - cos(\pi))⋅ℯ                   ⎛   4⎛\pi⎞      4⎛\pi⎞⎞  ⅈ⋅\pi⋅(x₀ + x₁)⎟ 
⎜──────────────────────────────── + ⎜sin ⎜───⎟ + cos ⎜───⎟⎟⋅ℯ               ⎟⋅
⎝               4                   ⎝    ⎝ 4 ⎠       ⎝ 4 ⎠⎠                 ⎠ 

                 
                 
 -ⅈ⋅\pi⋅(x₀ + x₁)
ℯ                
                 

In [40]:
# calculate P(|00>) = |<00|psi>|

p00 = simplify(re(braket00)*re(braket00)+im(braket00)*im(braket00))

display(p00)

                                                                              
                                                       ⎛                      
                                                       ⎜               ⎛   2  
              2    2⎛   2                          ⎞   ⎜-cos(\pi) - cos⎝\pi  -
(1 - cos(\pi)) ⋅sin ⎝\pi  - \pi⋅x₀ - \pi⋅x₁ + x₀⋅x₁⎠   ⎝                      
──────────────────────────────────────────────────── + ───────────────────────
                         16                                                   

                                                                              
                               ⎛     2                                ⎞      ⎛
                        ⎞   cos⎝- \pi  + \pi⋅x₀ + \pi⋅x₁ + \pi - x₀⋅x₁⎠   cos⎝
 \pi⋅x₀ - \pi⋅x₁ + x₀⋅x₁⎠ + ─────────────────────────────────────────── + ────
                                                 2                            
───────────────────────────────────────────────────

In [41]:
# derive expression for <01|psi>

raw01 = braket(idx_to_vect("01"), psi).expand(trig=True).expand()

braket01 = simplify(raw01, maxn=17, chop=True)
temp = braket01
for a in preorder_traversal(temp):
    if isinstance(a, Float):
        braket01 = braket01.subs(a, round(a, 15))

display(braket01)

  ⎛       ⎛   2        ⎞                ⎛   2        ⎞                        
  ⎜     ⅈ⋅⎝\pi  + x₀⋅x₁⎠    2⎛\pi⎞    ⅈ⋅⎝\pi  + x₀⋅x₁⎠    ⅈ⋅\pi⋅(x₀ + x₁)    ⎛
ⅈ⋅⎜- 2⋅ℯ                ⋅cos ⎜───⎟ + ℯ                 + ℯ               ⋅cos⎜
  ⎝                          ⎝ 4 ⎠                                           ⎝

    ⎞                                    
\pi⎞⎟  -ⅈ⋅\pi⋅(x₀ + x₁)    ⎛\pi⎞    ⎛\pi⎞
───⎟⎟⋅ℯ                ⋅sin⎜───⎟⋅cos⎜───⎟
 2 ⎠⎠                      ⎝ 4 ⎠    ⎝ 4 ⎠

In [42]:
# calculate P(|01>) = |<01|psi>|

p01 = simplify(re(braket01)*re(braket01)+im(braket01)*im(braket01))

display(p01)

⎛                                                        2                    
⎜               ⎛   ⎛   2                          ⎞    ⎞                     
⎜(cos(\pi) + 1)⋅⎝cos⎝\pi  - \pi⋅x₀ - \pi⋅x₁ + x₀⋅x₁⎠ - 1⎠       2⎛   2        
⎜───────────────────────────────────────────────────────── + sin ⎝\pi  - \pi⋅x
⎝                            2                                                

                             ⎞                    
                             ⎟                    
                  ⎞    2⎛\pi⎞⎟    2⎛\pi⎞    2⎛\pi⎞
₀ - \pi⋅x₁ + x₀⋅x₁⎠⋅cos ⎜───⎟⎟⋅sin ⎜───⎟⋅cos ⎜───⎟
                        ⎝ 2 ⎠⎠     ⎝ 4 ⎠     ⎝ 4 ⎠

In [43]:
p0 = simplify(p00+p01)

display(p0)

⎛                                                        2                    
⎜               ⎛   ⎛   2                          ⎞    ⎞                     
⎜(cos(\pi) + 1)⋅⎝cos⎝\pi  - \pi⋅x₀ - \pi⋅x₁ + x₀⋅x₁⎠ - 1⎠       2⎛   2        
⎜───────────────────────────────────────────────────────── + sin ⎝\pi  - \pi⋅x
⎝                            2                                                

                             ⎞                                                
                             ⎟                                     2    2⎛   2
                  ⎞    2⎛\pi⎞⎟    2⎛\pi⎞    2⎛\pi⎞   (cos(\pi) - 1) ⋅sin ⎝\pi 
₀ - \pi⋅x₁ + x₀⋅x₁⎠⋅cos ⎜───⎟⎟⋅sin ⎜───⎟⋅cos ⎜───⎟ + ─────────────────────────
                        ⎝ 2 ⎠⎠     ⎝ 4 ⎠     ⎝ 4 ⎠                            

                                                                              
                          ⎞   ⎛                  ⎛   2                        
 - \pi⋅x₀ - \pi⋅x₁ + x₀⋅x₁⎠   ⎝2⋅cos(\pi) + 2⋅cos⎝

In [47]:
pi_val = 3.141592653589
expression = sin(pi_val**2-pi*x0-pi*x1+x0*x1)**2
res = simplify(expression)

display(res)

   2                                            
sin (\pi⋅x₀ + \pi⋅x₁ - x₀⋅x₁ - 9.86960440108437)

# SimpleSA

## SVAO

In [8]:
t0 = symbols("\\theta_0", real=True)
t1 = symbols("\\theta_1", real=True)
t2 = symbols("\\theta_2", real=True)
pi = symbols('\\pi', real=True)

In [9]:
psi0 = idx_to_vect("0000")

psi1 = tfv(mtp2([H, H, id(1), H]), psi0)

psi2 = tfv(cgate2(x(), 0, 1), psi1)

psi3 = tfv(mtp2([id(1), rz(t0), id(1), id(1)]), psi2)

psi4 = tfv(mtp2([id(1), H, id(1), id(1)]), psi3)

psi5 = tfv(cgate2(x(), 1, 3), psi4)

psi6 = tfv(mtp2([id(1), id(1), id(1), rz(t1)]), psi5)

psi7 = tfv(mtp2([id(1), id(1), id(1), H]), psi6)

psi8 = tfv(mtp2([id(1), id(1), H, id(1)]), psi7)

psi9 = tfv(mtp2([id(1), id(1), rz(t2), id(1)]), psi8)

psi10 = tfv(cgate2(x(), 2, 1), psi9)

psi11 = tfv(mtp2([id(1), id(1), H, H]), psi10)

In [10]:
for e in psi11:
    display(e)

    ⎛      ⎛      ⅈ⋅\theta₀         -ⅈ⋅\theta₀ ⎞  ⅈ⋅\theta₁       ⎛      ⅈ⋅\th
    ⎜      ⎜      ─────────         ───────────⎟  ─────────       ⎜      ─────
    ⎜      ⎜          2                  2     ⎟      2           ⎜          2
0.5⋅⎝- 0.5⋅⎝0.25⋅ℯ          + 0.25⋅ℯ           ⎠⋅ℯ          + 0.5⋅⎝0.25⋅ℯ     

eta₀         -ⅈ⋅\theta₀ ⎞  -ⅈ⋅\theta₁ ⎞  ⅈ⋅\theta₂       ⎛      ⎛      ⅈ⋅\thet
────         ───────────⎟  ───────────⎟  ─────────       ⎜      ⎜      ───────
                  2     ⎟       2     ⎟      2           ⎜      ⎜          2  
     + 0.25⋅ℯ           ⎠⋅ℯ           ⎠⋅ℯ          + 0.5⋅⎝- 0.5⋅⎝0.25⋅ℯ       

a₀         -ⅈ⋅\theta₀ ⎞  ⅈ⋅\theta₁       ⎛      ⅈ⋅\theta₀         -ⅈ⋅\theta₀ ⎞
──         ───────────⎟  ─────────       ⎜      ─────────         ───────────⎟
                2     ⎟      2           ⎜          2                  2     ⎟
   + 0.25⋅ℯ           ⎠⋅ℯ          + 0.5⋅⎝0.25⋅ℯ          + 0.25⋅ℯ           ⎠

  -ⅈ⋅\theta₁ ⎞  -ⅈ⋅\theta₂        ⎛    ⎛      ⅈ⋅\

      ⎛      ⎛      ⅈ⋅\theta₀         -ⅈ⋅\theta₀ ⎞  ⅈ⋅\theta₁       ⎛      ⅈ⋅\
      ⎜      ⎜      ─────────         ───────────⎟  ─────────       ⎜      ───
      ⎜      ⎜          2                  2     ⎟      2           ⎜         
- 0.5⋅⎝- 0.5⋅⎝0.25⋅ℯ          + 0.25⋅ℯ           ⎠⋅ℯ          + 0.5⋅⎝0.25⋅ℯ   

theta₀         -ⅈ⋅\theta₀ ⎞  -ⅈ⋅\theta₁ ⎞  ⅈ⋅\theta₂       ⎛      ⎛      ⅈ⋅\th
──────         ───────────⎟  ───────────⎟  ─────────       ⎜      ⎜      ─────
 2                  2     ⎟       2     ⎟      2           ⎜      ⎜          2
       + 0.25⋅ℯ           ⎠⋅ℯ           ⎠⋅ℯ          - 0.5⋅⎝- 0.5⋅⎝0.25⋅ℯ     

eta₀         -ⅈ⋅\theta₀ ⎞  ⅈ⋅\theta₁       ⎛      ⅈ⋅\theta₀         -ⅈ⋅\theta₀
────         ───────────⎟  ─────────       ⎜      ─────────         ──────────
                  2     ⎟      2           ⎜          2                  2    
     + 0.25⋅ℯ           ⎠⋅ℯ          + 0.5⋅⎝0.25⋅ℯ          + 0.25⋅ℯ          

 ⎞  -ⅈ⋅\theta₁ ⎞  -ⅈ⋅\theta₂        ⎛    ⎛      ⅈ

      ⎛      ⎛      ⅈ⋅\theta₀         -ⅈ⋅\theta₀ ⎞  ⅈ⋅\theta₁       ⎛      ⅈ⋅\
      ⎜      ⎜      ─────────         ───────────⎟  ─────────       ⎜      ───
      ⎜      ⎜          2                  2     ⎟      2           ⎜         
- 0.5⋅⎝- 0.5⋅⎝0.25⋅ℯ          + 0.25⋅ℯ           ⎠⋅ℯ          + 0.5⋅⎝0.25⋅ℯ   

theta₀         -ⅈ⋅\theta₀ ⎞  -ⅈ⋅\theta₁ ⎞  ⅈ⋅\theta₂       ⎛      ⎛      ⅈ⋅\th
──────         ───────────⎟  ───────────⎟  ─────────       ⎜      ⎜      ─────
 2                  2     ⎟       2     ⎟      2           ⎜      ⎜          2
       + 0.25⋅ℯ           ⎠⋅ℯ           ⎠⋅ℯ          + 0.5⋅⎝- 0.5⋅⎝0.25⋅ℯ     

eta₀         -ⅈ⋅\theta₀ ⎞  ⅈ⋅\theta₁       ⎛      ⅈ⋅\theta₀         -ⅈ⋅\theta₀
────         ───────────⎟  ─────────       ⎜      ─────────         ──────────
                  2     ⎟      2           ⎜          2                  2    
     + 0.25⋅ℯ           ⎠⋅ℯ          + 0.5⋅⎝0.25⋅ℯ          + 0.25⋅ℯ          

 ⎞  -ⅈ⋅\theta₁ ⎞  -ⅈ⋅\theta₂        ⎛    ⎛      ⅈ

    ⎛      ⎛      ⅈ⋅\theta₀         -ⅈ⋅\theta₀ ⎞  ⅈ⋅\theta₁       ⎛      ⅈ⋅\th
    ⎜      ⎜      ─────────         ───────────⎟  ─────────       ⎜      ─────
    ⎜      ⎜          2                  2     ⎟      2           ⎜          2
0.5⋅⎝- 0.5⋅⎝0.25⋅ℯ          + 0.25⋅ℯ           ⎠⋅ℯ          + 0.5⋅⎝0.25⋅ℯ     

eta₀         -ⅈ⋅\theta₀ ⎞  -ⅈ⋅\theta₁ ⎞  ⅈ⋅\theta₂       ⎛      ⎛      ⅈ⋅\thet
────         ───────────⎟  ───────────⎟  ─────────       ⎜      ⎜      ───────
                  2     ⎟       2     ⎟      2           ⎜      ⎜          2  
     + 0.25⋅ℯ           ⎠⋅ℯ           ⎠⋅ℯ          - 0.5⋅⎝- 0.5⋅⎝0.25⋅ℯ       

a₀         -ⅈ⋅\theta₀ ⎞  ⅈ⋅\theta₁       ⎛      ⅈ⋅\theta₀         -ⅈ⋅\theta₀ ⎞
──         ───────────⎟  ─────────       ⎜      ─────────         ───────────⎟
                2     ⎟      2           ⎜          2                  2     ⎟
   + 0.25⋅ℯ           ⎠⋅ℯ          + 0.5⋅⎝0.25⋅ℯ          + 0.25⋅ℯ           ⎠

  -ⅈ⋅\theta₁ ⎞  -ⅈ⋅\theta₂        ⎛    ⎛      ⅈ⋅\

    ⎛      ⎛        ⅈ⋅\theta₀         -ⅈ⋅\theta₀ ⎞  ⅈ⋅\theta₁       ⎛        ⅈ
    ⎜      ⎜        ─────────         ───────────⎟  ─────────       ⎜        ─
    ⎜      ⎜            2                  2     ⎟      2           ⎜         
0.5⋅⎝- 0.5⋅⎝- 0.25⋅ℯ          + 0.25⋅ℯ           ⎠⋅ℯ          + 0.5⋅⎝- 0.25⋅ℯ 

⋅\theta₀         -ⅈ⋅\theta₀ ⎞  -ⅈ⋅\theta₁ ⎞  ⅈ⋅\theta₂       ⎛      ⎛        ⅈ
────────         ───────────⎟  ───────────⎟  ─────────       ⎜      ⎜        ─
   2                  2     ⎟       2     ⎟      2           ⎜      ⎜         
         + 0.25⋅ℯ           ⎠⋅ℯ           ⎠⋅ℯ          + 0.5⋅⎝- 0.5⋅⎝- 0.25⋅ℯ 

⋅\theta₀         -ⅈ⋅\theta₀ ⎞  ⅈ⋅\theta₁       ⎛        ⅈ⋅\theta₀         -ⅈ⋅\
────────         ───────────⎟  ─────────       ⎜        ─────────         ────
   2                  2     ⎟      2           ⎜            2                 
         + 0.25⋅ℯ           ⎠⋅ℯ          + 0.5⋅⎝- 0.25⋅ℯ          + 0.25⋅ℯ    

theta₀ ⎞  -ⅈ⋅\theta₁ ⎞  -ⅈ⋅\theta₂        ⎛    ⎛ 

...

    ⎛      ⎛      ⅈ⋅\theta₀         -ⅈ⋅\theta₀ ⎞  ⅈ⋅\theta₁       ⎛      ⅈ⋅\th
    ⎜      ⎜      ─────────         ───────────⎟  ─────────       ⎜      ─────
    ⎜      ⎜          2                  2     ⎟      2           ⎜          2
0.5⋅⎝- 0.5⋅⎝0.25⋅ℯ          + 0.25⋅ℯ           ⎠⋅ℯ          + 0.5⋅⎝0.25⋅ℯ     

eta₀         -ⅈ⋅\theta₀ ⎞  -ⅈ⋅\theta₁ ⎞  ⅈ⋅\theta₂       ⎛      ⎛      ⅈ⋅\thet
────         ───────────⎟  ───────────⎟  ─────────       ⎜      ⎜      ───────
                  2     ⎟       2     ⎟      2           ⎜      ⎜          2  
     + 0.25⋅ℯ           ⎠⋅ℯ           ⎠⋅ℯ          - 0.5⋅⎝- 0.5⋅⎝0.25⋅ℯ       

a₀         -ⅈ⋅\theta₀ ⎞  ⅈ⋅\theta₁       ⎛      ⅈ⋅\theta₀         -ⅈ⋅\theta₀ ⎞
──         ───────────⎟  ─────────       ⎜      ─────────         ───────────⎟
                2     ⎟      2           ⎜          2                  2     ⎟
   + 0.25⋅ℯ           ⎠⋅ℯ          + 0.5⋅⎝0.25⋅ℯ          + 0.25⋅ℯ           ⎠

  -ⅈ⋅\theta₁ ⎞  -ⅈ⋅\theta₂        ⎛    ⎛      ⅈ⋅\

    ⎛      ⎛        ⅈ⋅\theta₀         -ⅈ⋅\theta₀ ⎞  ⅈ⋅\theta₁       ⎛        ⅈ
    ⎜      ⎜        ─────────         ───────────⎟  ─────────       ⎜        ─
    ⎜      ⎜            2                  2     ⎟      2           ⎜         
0.5⋅⎝- 0.5⋅⎝- 0.25⋅ℯ          + 0.25⋅ℯ           ⎠⋅ℯ          + 0.5⋅⎝- 0.25⋅ℯ 

⋅\theta₀         -ⅈ⋅\theta₀ ⎞  -ⅈ⋅\theta₁ ⎞  ⅈ⋅\theta₂       ⎛      ⎛        ⅈ
────────         ───────────⎟  ───────────⎟  ─────────       ⎜      ⎜        ─
   2                  2     ⎟       2     ⎟      2           ⎜      ⎜         
         + 0.25⋅ℯ           ⎠⋅ℯ           ⎠⋅ℯ          + 0.5⋅⎝- 0.5⋅⎝- 0.25⋅ℯ 

⋅\theta₀         -ⅈ⋅\theta₀ ⎞  ⅈ⋅\theta₁       ⎛        ⅈ⋅\theta₀         -ⅈ⋅\
────────         ───────────⎟  ─────────       ⎜        ─────────         ────
   2                  2     ⎟      2           ⎜            2                 
         + 0.25⋅ℯ           ⎠⋅ℯ          + 0.5⋅⎝- 0.25⋅ℯ          + 0.25⋅ℯ    

theta₀ ⎞  -ⅈ⋅\theta₁ ⎞  -ⅈ⋅\theta₂        ⎛    ⎛ 

      ⎛      ⎛        ⅈ⋅\theta₀         -ⅈ⋅\theta₀ ⎞  ⅈ⋅\theta₁       ⎛       
      ⎜      ⎜        ─────────         ───────────⎟  ─────────       ⎜       
      ⎜      ⎜            2                  2     ⎟      2           ⎜       
- 0.5⋅⎝- 0.5⋅⎝- 0.25⋅ℯ          + 0.25⋅ℯ           ⎠⋅ℯ          + 0.5⋅⎝- 0.25⋅

 ⅈ⋅\theta₀         -ⅈ⋅\theta₀ ⎞  -ⅈ⋅\theta₁ ⎞  ⅈ⋅\theta₂       ⎛      ⎛       
 ─────────         ───────────⎟  ───────────⎟  ─────────       ⎜      ⎜       
     2                  2     ⎟       2     ⎟      2           ⎜      ⎜       
ℯ          + 0.25⋅ℯ           ⎠⋅ℯ           ⎠⋅ℯ          - 0.5⋅⎝- 0.5⋅⎝- 0.25⋅

 ⅈ⋅\theta₀         -ⅈ⋅\theta₀ ⎞  ⅈ⋅\theta₁       ⎛        ⅈ⋅\theta₀         -ⅈ
 ─────────         ───────────⎟  ─────────       ⎜        ─────────         ──
     2                  2     ⎟      2           ⎜            2               
ℯ          + 0.25⋅ℯ           ⎠⋅ℯ          + 0.5⋅⎝- 0.25⋅ℯ          + 0.25⋅ℯ  

⋅\theta₀ ⎞  -ⅈ⋅\theta₁ ⎞  -ⅈ⋅\theta₂        ⎛    

      ⎛      ⎛        ⅈ⋅\theta₀         -ⅈ⋅\theta₀ ⎞  ⅈ⋅\theta₁       ⎛       
      ⎜      ⎜        ─────────         ───────────⎟  ─────────       ⎜       
      ⎜      ⎜            2                  2     ⎟      2           ⎜       
- 0.5⋅⎝- 0.5⋅⎝- 0.25⋅ℯ          + 0.25⋅ℯ           ⎠⋅ℯ          + 0.5⋅⎝- 0.25⋅

 ⅈ⋅\theta₀         -ⅈ⋅\theta₀ ⎞  -ⅈ⋅\theta₁ ⎞  ⅈ⋅\theta₂       ⎛      ⎛       
 ─────────         ───────────⎟  ───────────⎟  ─────────       ⎜      ⎜       
     2                  2     ⎟       2     ⎟      2           ⎜      ⎜       
ℯ          + 0.25⋅ℯ           ⎠⋅ℯ           ⎠⋅ℯ          + 0.5⋅⎝- 0.5⋅⎝- 0.25⋅

 ⅈ⋅\theta₀         -ⅈ⋅\theta₀ ⎞  ⅈ⋅\theta₁       ⎛        ⅈ⋅\theta₀         -ⅈ
 ─────────         ───────────⎟  ─────────       ⎜        ─────────         ──
     2                  2     ⎟      2           ⎜            2               
ℯ          + 0.25⋅ℯ           ⎠⋅ℯ          + 0.5⋅⎝- 0.25⋅ℯ          + 0.25⋅ℯ  

⋅\theta₀ ⎞  -ⅈ⋅\theta₁ ⎞  -ⅈ⋅\theta₂        ⎛    

    ⎛      ⎛        ⅈ⋅\theta₀         -ⅈ⋅\theta₀ ⎞  ⅈ⋅\theta₁       ⎛        ⅈ
    ⎜      ⎜        ─────────         ───────────⎟  ─────────       ⎜        ─
    ⎜      ⎜            2                  2     ⎟      2           ⎜         
0.5⋅⎝- 0.5⋅⎝- 0.25⋅ℯ          + 0.25⋅ℯ           ⎠⋅ℯ          + 0.5⋅⎝- 0.25⋅ℯ 

⋅\theta₀         -ⅈ⋅\theta₀ ⎞  -ⅈ⋅\theta₁ ⎞  ⅈ⋅\theta₂       ⎛      ⎛        ⅈ
────────         ───────────⎟  ───────────⎟  ─────────       ⎜      ⎜        ─
   2                  2     ⎟       2     ⎟      2           ⎜      ⎜         
         + 0.25⋅ℯ           ⎠⋅ℯ           ⎠⋅ℯ          - 0.5⋅⎝- 0.5⋅⎝- 0.25⋅ℯ 

⋅\theta₀         -ⅈ⋅\theta₀ ⎞  ⅈ⋅\theta₁       ⎛        ⅈ⋅\theta₀         -ⅈ⋅\
────────         ───────────⎟  ─────────       ⎜        ─────────         ────
   2                  2     ⎟      2           ⎜            2                 
         + 0.25⋅ℯ           ⎠⋅ℯ          + 0.5⋅⎝- 0.25⋅ℯ          + 0.25⋅ℯ    

theta₀ ⎞  -ⅈ⋅\theta₁ ⎞  -ⅈ⋅\theta₂        ⎛    ⎛ 

In [11]:
# derive expression for <0000|psi10>

raw0000 = braket(idx_to_vect("0000"), psi11).expand(trig=True).expand()

braket0000 = simplify(raw0000, maxn=17, chop=True)
temp = braket0000
for a in preorder_traversal(temp):
    if isinstance(a, Float):
        braket0000 = braket0000.subs(a, round(a, 15))

# display(braket0000)

In [12]:
# calculate P(|0000>) = |<0000|psi10>|

p0000 = re(braket0000)*re(braket0000)+im(braket0000)*im(braket0000)

display(p0000)

                                                                              
         ⎛                                                          ⎛\theta₀  
0.015625⋅⎜(sin(\theta₀) + sin(\theta₂) + sin(\theta₀ + \theta₂))⋅sin⎜─────── +
         ⎝                                                          ⎝   2     

                                                                              
 \theta₁   \theta₂⎞                                                           
 ─────── + ───────⎟ + (cos(\theta₀) + cos(\theta₂) + cos(\theta₀ + \theta₂) + 
    2         2   ⎠                                                           

                                    2                                         
      ⎛\theta₀   \theta₁   \theta₂⎞⎞             ⎛                            
1)⋅cos⎜─────── + ─────── + ───────⎟⎟  + 0.015625⋅⎜(sin(\theta₀) + sin(\theta₂)
      ⎝   2         2         2   ⎠⎠             ⎝                            

                                                 

In [13]:
# derive expression for <0100|psi10>

raw0100 = braket(idx_to_vect("0100"), psi11).expand(trig=True).expand()

braket0100 = simplify(raw0100, maxn=17, chop=True)
temp = braket0100
for a in preorder_traversal(temp):
    if isinstance(a, Float):
        braket0100 = braket0100.subs(a, round(a, 15))

# display(braket0100)

In [14]:
# calculate P(|0100>) = |<0100|psi10>|

p0100 = re(braket0100)*re(braket0100)+im(braket0100)*im(braket0100)

display(p0100)

                                                                              
⎛                                                                             
⎜(-0.125⋅sin(\theta₀) + 0.125⋅sin(\theta₂) - 0.125⋅sin(\theta₀ + \theta₂))⋅sin
⎝                                                                             

                                                                              
⎛\theta₀   \theta₁   \theta₂⎞                                                 
⎜─────── + ─────── + ───────⎟ + (-0.125⋅cos(\theta₀) + 0.125⋅cos(\theta₂) - 0.
⎝   2         2         2   ⎠                                                 

                                                                     2        
                                       ⎛\theta₀   \theta₁   \theta₂⎞⎞    ⎛    
125⋅cos(\theta₀ + \theta₂) + 0.125)⋅cos⎜─────── + ─────── + ───────⎟⎟  + ⎜(-0.
                                       ⎝   2         2         2   ⎠⎠    ⎝    

                                                 

In [15]:
# calculate P(O=|0>) = P(|0000>)/( P(|0000>)+P(|0100>) )

p0 = simplify(p0000/(p0000+p0100))

display(p0)

0.5⋅cos(\theta₀) + 0.5

In [19]:
display(braket0000)

                                                              -ⅈ⋅(\theta₀ + \t
                                                              ────────────────
      ⎛ ⅈ⋅\theta₀    ⅈ⋅\theta₂    ⅈ⋅(\theta₀ + \theta₂)    ⎞                  
0.125⋅⎝ℯ          + ℯ          + ℯ                      + 1⎠⋅ℯ                

heta₁ + \theta₂) 
─────────────────
2                
                 

In [20]:
display(p0000)

                                                                              
         ⎛                                                          ⎛\theta₀  
0.015625⋅⎜(sin(\theta₀) + sin(\theta₂) + sin(\theta₀ + \theta₂))⋅sin⎜─────── +
         ⎝                                                          ⎝   2     

                                                                              
 \theta₁   \theta₂⎞                                                           
 ─────── + ───────⎟ + (cos(\theta₀) + cos(\theta₂) + cos(\theta₀ + \theta₂) + 
    2         2   ⎠                                                           

                                    2                                         
      ⎛\theta₀   \theta₁   \theta₂⎞⎞             ⎛                            
1)⋅cos⎜─────── + ─────── + ───────⎟⎟  + 0.015625⋅⎜(sin(\theta₀) + sin(\theta₂)
      ⎝   2         2         2   ⎠⎠             ⎝                            

                                                 

In [27]:
display(simplify(p0000))

0.0625⋅cos(\theta₀)⋅cos(\theta₂) + 0.0625⋅cos(\theta₀) + 0.0625⋅cos(\theta₂) +
 0.0625

In [21]:
display(braket0100)

                                                                              
                                                                              
⎛         ⅈ⋅\theta₀          ⅈ⋅\theta₂          ⅈ⋅(\theta₀ + \theta₂)        ⎞
⎝- 0.125⋅ℯ          + 0.125⋅ℯ          - 0.125⋅ℯ                      + 0.125⎠

  -ⅈ⋅(\theta₀ + \theta₁ + \theta₂) 
  ─────────────────────────────────
                  2                
⋅ℯ                                 

In [29]:
display(p0100)

                                                                              
⎛                                                                             
⎜(-0.125⋅sin(\theta₀) + 0.125⋅sin(\theta₂) - 0.125⋅sin(\theta₀ + \theta₂))⋅sin
⎝                                                                             

                                                                              
⎛\theta₀   \theta₁   \theta₂⎞                                                 
⎜─────── + ─────── + ───────⎟ + (-0.125⋅cos(\theta₀) + 0.125⋅cos(\theta₂) - 0.
⎝   2         2         2   ⎠                                                 

                                                                     2        
                                       ⎛\theta₀   \theta₁   \theta₂⎞⎞    ⎛    
125⋅cos(\theta₀ + \theta₂) + 0.125)⋅cos⎜─────── + ─────── + ───────⎟⎟  + ⎜(-0.
                                       ⎝   2         2         2   ⎠⎠    ⎝    

                                                 

In [31]:
s0000 = simplify(p0000)
s0100 = simplify(p0100)

In [33]:
(s0000)/(s0000+s0100)

0.0625⋅cos(\theta₀)⋅cos(\theta₂) + 0.0625⋅cos(\theta₀) + 0.0625⋅cos(\theta₂) +
──────────────────────────────────────────────────────────────────────────────
                              0.125⋅cos(\theta₂) + 0.125                      

 0.0625
───────
       

# H-CRx

## SVAO

In [6]:
t0 = symbols("\\theta_0", real=True)
t1 = symbols("\\theta_1", real=True)
t2 = symbols("\\theta_2", real=True)
t3 = symbols("\\theta_3", real=True)
t4 = symbols("\\theta_4", real=True)
pi = symbols('\\pi', real=True)

In [7]:
# |psi0> = |0000>
psi0 = idx_to_vect("0000")

# |psi1> = H*H*I*H |psi0>
psi1 = tfv(mtp2([H, H, id(1), H]), psi0)

# |psi2> = CRx(t0)[0, 3] |psi1>
psi2 = tfv(cgate2(rx(t0), 0, 3), psi1)

# |psi3> = CRx(t1)[3, 1] |psi2>
psi3 = tfv(cgate2(rx(t1), 3, 1), psi2)

# |psi4> = CRx(t2)[1, 0]*I^2 |psi3>
psi4 = tfv(mtp2([cgate2(rx(t2), 1, 0), id(1), id(1)]), psi3)

# |psi5> = H*H*I*H |psi4>
psi5 = tfv(mtp2([H, H, id(1), H]), psi4)

# |psi6> = CRx(t0)[0, 1]*I^2 |psi5>
psi6 = tfv(mtp2([cgate2(rx(t0), 0, 1), id(1), id(1)]), psi5)

# |psi7> = CRx(t1)[1, 3] |psi6>
psi7 = tfv(cgate2(rx(t1), 1, 3), psi6)

# |psi8> = CRx(t2)[3, 0] |psi7>
psi8 = tfv(cgate2(rx(t2), 3, 0), psi7)

# |psi9> = CRx(t4)[2, 3] |psi8>
psi9 = tfv(cgate2(rx(t4), 2, 3), psi8)

# |psi10> = CNOT[3, 2] |psi9>
psi10 = tfv(cgate2(rx(t3), 3, 2), psi9)

# |psi11> = I^2*H*H |psi10>
psi11 = tfv(mtp2([id(1), id(1), H, H]), psi10)

# |psi12> = CRx(t4)[2, 3] |psi11>
psi12 = tfv(cgate2(rx(t4), 2, 3), psi11)

# |psi13> = CRx(t3)[3, 2] |psi12>
psi13 = tfv(cgate2(rx(t3), 3, 2), psi12)

# |psi14> = I^2*H*H |psi13>
psi14 = tfv(mtp2([id(1), id(1), H, H]), psi13)

In [8]:
# derive expression for <0000|psi10>

raw0000 = braket(idx_to_vect("0000"), psi14).expand(trig=True).expand()

braket0000 = simplify(raw0000, maxn=17, chop=True)
temp = braket0000
for a in preorder_traversal(temp):
    if isinstance(a, Float):
        braket0000 = braket0000.subs(a, round(a, 15))

# display(braket0000)

In [9]:
# calculate P(|0000>) = |<0000|psi10>|

p0000 = re(braket0000)*re(braket0000)+im(braket0000)*im(braket0000)

display(p0000)

                                                                              
         ⎛     2⎛\theta₀⎞    ⎛\theta₂⎞    2⎛\theta₃⎞          2⎛\theta₀⎞    ⎛\
0.015625⋅⎜- sin ⎜───────⎟⋅sin⎜───────⎟⋅sin ⎜───────⎟ + 0.5⋅sin ⎜───────⎟⋅sin⎜─
         ⎝      ⎝   2   ⎠    ⎝   2   ⎠     ⎝   2   ⎠           ⎝   2   ⎠    ⎝ 

                                                                              
theta₂⎞      ⎛\theta₀⎞    ⎛\theta₂⎞    ⎛\theta₃⎞    ⎛\theta₀   \theta₁   \thet
──────⎟ - sin⎜───────⎟⋅sin⎜───────⎟⋅sin⎜───────⎟⋅cos⎜─────── + ─────── + ─────
  2   ⎠      ⎝   2   ⎠    ⎝   2   ⎠    ⎝   2   ⎠    ⎝   2         2         2 

                                                                              
a₃⎞          ⎛\theta₀⎞    ⎛\theta₂⎞    ⎛\theta₀   \theta₁⎞          ⎛\theta₀⎞ 
──⎟ - 0.5⋅sin⎜───────⎟⋅sin⎜───────⎟⋅sin⎜─────── + ───────⎟ - 0.5⋅sin⎜───────⎟⋅
  ⎠          ⎝   2   ⎠    ⎝   2   ⎠    ⎝   2         2   ⎠          ⎝   2   ⎠ 

                                                 

In [10]:
# derive expression for <0100|psi10>

raw0100 = braket(idx_to_vect("0100"), psi14).expand(trig=True).expand()

braket0100 = simplify(raw0100, maxn=17, chop=True)
temp = braket0100
for a in preorder_traversal(temp):
    if isinstance(a, Float):
        braket0100 = braket0100.subs(a, round(a, 15))

# display(braket0100)

In [11]:
# calculate P(|0100>) = |<0100|psi10>|

p0100 = re(braket0100)*re(braket0100)+im(braket0100)*im(braket0100)

display(p0100)

                                                                              
         ⎛     ⎛\theta₀⎞    ⎛\theta₂⎞    ⎛\theta₃⎞    ⎛\theta₀   \theta₃⎞     
0.015625⋅⎜- sin⎜───────⎟⋅sin⎜───────⎟⋅sin⎜───────⎟⋅sin⎜─────── + ───────⎟ + si
         ⎝     ⎝   2   ⎠    ⎝   2   ⎠    ⎝   2   ⎠    ⎝   2         2   ⎠     

                                                                              
 ⎛\theta₀⎞    ⎛\theta₂⎞    ⎛\theta₃⎞    ⎛\theta₀   \theta₁   \theta₃⎞         
n⎜───────⎟⋅sin⎜───────⎟⋅sin⎜───────⎟⋅sin⎜─────── + ─────── + ───────⎟ + 0.5⋅si
 ⎝   2   ⎠    ⎝   2   ⎠    ⎝   2   ⎠    ⎝   2         2         2   ⎠         

                                                                              
 ⎛\theta₀⎞    ⎛\theta₂⎞    ⎛\theta₀⎞          ⎛\theta₀⎞    ⎛\theta₂⎞    ⎛\thet
n⎜───────⎟⋅sin⎜───────⎟⋅cos⎜───────⎟ - 0.5⋅sin⎜───────⎟⋅sin⎜───────⎟⋅cos⎜─────
 ⎝   2   ⎠    ⎝   2   ⎠    ⎝   2   ⎠          ⎝   2   ⎠    ⎝   2   ⎠    ⎝   2 

                                                 

In [12]:
p0_denom = 64*(p0000+p0100)

# display(p0_denom)

In [13]:
# calculate P(O=|0>) = P(|0000>)/( P(|0000>)+P(|0100>) )

p0 = 64*p0000/p0_denom

display(p0)

                                                                              
                                                                              
                                                                              
                                                                              
──────────────────────────────────────────────────────────────────────────────
                                                                              
⎛     ⎛\theta₀⎞    ⎛\theta₂⎞    ⎛\theta₃⎞    ⎛\theta₀   \theta₃⎞      ⎛\theta₀
⎜- sin⎜───────⎟⋅sin⎜───────⎟⋅sin⎜───────⎟⋅sin⎜─────── + ───────⎟ + sin⎜───────
⎝     ⎝   2   ⎠    ⎝   2   ⎠    ⎝   2   ⎠    ⎝   2         2   ⎠      ⎝   2   

                                                                              
                                                                              
                                                                              
                                                   

In [14]:
tau = 2*3.141592653589793

verb_0 = tau*3.1103484691146415
verb_1 = tau*1.2336966690507978
verb_2 = tau*0.3492496431368752
adj_0 = tau*0.9
adj_1 = tau*0.5

In [15]:
out0000 = simplify(p0.subs([(t0, verb_0), (t1, verb_1), (t2, verb_2)]))

35 mins

In [16]:
temp = out0000
for a in preorder_traversal(temp):
    if isinstance(a, Float):
        out0000 = out0000.subs(a, round(a, 14))

out0000

                                                                              
                                                                              
                                                                              
                                                                              
──────────────────────────────────────────────────────────────────────────────
                                                                              
                                                                              
                                                                              
0.1903798020997⋅sin(\theta₃) - 0.1903798020997⋅sin(\theta₄) - 0.04759495052492
                                                                              
                                                                              
                                                                              

                                                   

In [17]:
p0.subs([(t0, verb_0), (t1, verb_1), (t2, verb_2), (t3, adj_0), (t4, adj_1)])

0.434679059059182

In [18]:
sum = 0
minval = 1
maxval = 0
n = 6
bins = [i/(2**n) for i in range(2**n+1)]
log = []

v_0 = 0.25
v_1 = 0.125
v_2 = 0.5

for a_0 in bins:
    for a_1 in bins:
        if p0000.subs([(t0, v_0*tau), (t1, v_1*tau), (t2, v_2*tau), (t3, a_0*tau), (t4, a_1*tau)]) > 0:
            val = p0.subs([(t0, v_0*tau), (t1, v_1*tau), (t2, v_2*tau), (t3, a_0*tau), (t4, a_1*tau)])
            sum += val
            log.append(val)

            if val > maxval:
                maxval = val
                
            if val < minval:
                minval = val
        else:
            minval = 0
            log.append(0)

In [22]:
log_freq = {}

for l_ in sorted(log):
    # l = int((int(l_*100)+1)/10)/10
    l = int(l_*100)/10
    if l in log_freq:
        log_freq[l] += 1
    else:
        log_freq[l] = 1

for key in list(log_freq.keys()):
    print(f"{key}: {log_freq[key]}")

0.0: 133
0.1: 19
0.2: 12
0.3: 15
0.4: 29
0.5: 37
0.6: 12
0.7: 8
0.8: 8
0.9: 7
1.0: 8
1.1: 9
1.2: 8
1.3: 10
1.4: 8
1.5: 11
1.6: 9
1.7: 9
1.8: 10
1.9: 8
2.0: 10
2.1: 8
2.2: 11
2.3: 10
2.4: 13
2.5: 8
2.6: 12
2.7: 12
2.8: 16
2.9: 7
3.0: 11
3.1: 7
3.2: 9
3.3: 7
3.4: 11
3.5: 8
3.6: 13
3.7: 9
3.8: 12
3.9: 15
4.0: 5
4.1: 13
4.2: 16
4.3: 13
4.4: 16
4.5: 15
4.6: 23
4.7: 20
4.8: 27
4.9: 1099
5.0: 474
5.1: 240
5.2: 185
5.3: 151
5.4: 120
5.5: 105
5.6: 93
5.7: 90
5.8: 75
5.9: 66
6.0: 71
6.1: 55
6.2: 55
6.3: 48
6.4: 44
6.5: 39
6.6: 39
6.7: 30
6.8: 37
6.9: 27
7.0: 24
7.1: 27
7.2: 20
7.3: 28
7.4: 16
7.5: 16
7.6: 19
7.7: 15
7.8: 17
7.9: 14
8.0: 11
8.1: 15
8.2: 9
8.3: 12
8.4: 11
8.5: 13
8.6: 7
8.7: 11
8.8: 8
8.9: 7
9.0: 5
9.1: 7
9.2: 6
9.3: 5
9.4: 7
9.5: 4
9.6: 5
9.7: 9
9.8: 3
9.9: 4


In [61]:
latex = "\frac{1.38033500268611 \left(0.04572540985655 \sin{\left(\frac{\theta_{3}}{2} \right)} - 0.08738445212883 \sin{\left(\theta_{3} \right)} + 0.04572540985655 \sin{\left(\frac{\theta_{4}}{2} \right)} - 0.12904349440111 \sin{\left(\frac{\theta_{3}}{2} + \frac{\theta_{4}}{2} \right)} - \cos{\left(\frac{\theta_{3}}{2} \right)} + 0.92032535365862 \cos{\left(\theta_{3} \right)} + 0.84065070731724 \cos{\left(\frac{\theta_{4}}{2} \right)} - 1.0 \cos{\left(\frac{\theta_{3}}{2} + \frac{\theta_{4}}{2} \right)} - 0.07967464634138\right)^{2} + 1.38033500268611 \left(\sin{\left(\frac{\theta_{3}}{2} \right)} - 0.92032535365862 \sin{\left(\theta_{3} \right)} + 1.0 \sin{\left(\frac{\theta_{4}}{2} \right)} - 0.84065070731724 \sin{\left(\frac{\theta_{3}}{2} + \frac{\theta_{4}}{2} \right)} + 0.04572540985655 \cos{\left(\frac{\theta_{3}}{2} \right)} - 0.08738445212883 \cos{\left(\theta_{3} \right)} - 0.12904349440111 \cos{\left(\frac{\theta_{4}}{2} \right)} + 0.04572540985655 \cos{\left(\frac{\theta_{3}}{2} + \frac{\theta_{4}}{2} \right)} - 0.04165904227228\right)^{2}}{0.1903798020997 \sin{\left(\theta_{3} \right)} - 0.1903798020997 \sin{\left(\theta_{4} \right)} - 0.04759495052492 \sin{\left(\frac{\theta_{3}}{2} - \frac{\theta_{4}}{2} \right)} + 0.04759495052492 \sin{\left(\frac{\theta_{3}}{2} + \frac{\theta_{4}}{2} \right)} + 0.1903798020997 \sin{\left(\theta_{3} - \frac{\theta_{4}}{2} \right)} + 0.1903798020997 \sin{\left(\theta_{3} + \frac{\theta_{4}}{2} \right)} + 0.1903798020997 \sin{\left(\theta_{3} + \theta_{4} \right)} - 0.04759495052492 \sin{\left(\frac{3 \theta_{3}}{2} - \frac{\theta_{4}}{2} \right)} + 0.04759495052492 \sin{\left(\frac{3 \theta_{3}}{2} + \frac{\theta_{4}}{2} \right)} - 10.8470314733245 \cos{\left(\frac{\theta_{3}}{2} \right)} - 0.33911855071498 \cos{\left(\theta_{3} \right)} + 0.06130376839315 \cos{\left(\frac{\theta_{4}}{2} \right)} - 0.33911855071498 \cos{\left(\theta_{4} \right)} + 0.03065188419658 \cos{\left(\frac{\theta_{3}}{2} - \frac{\theta_{4}}{2} \right)} - 5.42351573666224 \cos{\left(\frac{\theta_{3}}{2} + \frac{\theta_{4}}{2} \right)} - 0.33911855071498 \cos{\left(\theta_{3} - \frac{\theta_{4}}{2} \right)} + 11.2474537924326 \cos{\left(\theta_{3} + \frac{\theta_{4}}{2} \right)} + 0.33911855071498 \cos{\left(\theta_{3} + \theta_{4} \right)} - 5.45416762085882 \cos{\left(\frac{3 \theta_{3}}{2} + \frac{\theta_{4}}{2} \right)} + 11.3087575608258 - \frac{0.02379747526246 \cos{\left(2 \theta_{3} - \frac{\theta_{4}}{2} \right)}}{\sin{\left(\frac{\theta_{3}}{2} \right)}} + \frac{0.02379747526246 \cos{\left(2 \theta_{3} + \frac{\theta_{4}}{2} \right)}}{\sin{\left(\frac{\theta_{3}}{2} \right)}}}"

latex = latex.replace("\left", "").replace("\right", "")

latex = latex.replace("\f", "f")

latex = latex.replace("\theta_", "t").replace("\sin", "sin").replace("\cos", "cos")

for i in range(10):
    latex = latex.replace("t{"+str(i)+"}", "t"+str(i))

latex = latex.replace("}{", "}/{").replace("frac", "").replace("{(", "{").replace(")}", "}").replace("{", "(").replace("}", ")").replace(" ", "")

latex = latex.replace("t3", "x").replace("t4", "y")

latex = latex.replace("(2)", "2").replace("(x)", "x").replace("(y)", "y")

In [62]:
latex

'(1.38033500268611(0.04572540985655sin(x/2)-0.08738445212883sinx+0.04572540985655sin(y/2)-0.12904349440111sin(x/2+y/2)-cos(x/2)+0.92032535365862cosx+0.84065070731724cos(y/2)-1.0cos(x/2+y/2)-0.07967464634138)^2+1.38033500268611(sin(x/2)-0.92032535365862sinx+1.0sin(y/2)-0.84065070731724sin(x/2+y/2)+0.04572540985655cos(x/2)-0.08738445212883cosx-0.12904349440111cos(y/2)+0.04572540985655cos(x/2+y/2)-0.04165904227228)^2)/(0.1903798020997sinx-0.1903798020997siny-0.04759495052492sin(x/2-y/2)+0.04759495052492sin(x/2+y/2)+0.1903798020997sin(x-y/2)+0.1903798020997sin(x+y/2)+0.1903798020997sin(x+y)-0.04759495052492sin((3x)/2-y/2)+0.04759495052492sin((3x)/2+y/2)-10.8470314733245cos(x/2)-0.33911855071498cosx+0.06130376839315cos(y/2)-0.33911855071498cosy+0.03065188419658cos(x/2-y/2)-5.42351573666224cos(x/2+y/2)-0.33911855071498cos(x-y/2)+11.2474537924326cos(x+y/2)+0.33911855071498cos(x+y)-5.45416762085882cos((3x)/2+y/2)+11.3087575608258-(0.02379747526246cos(2x-y/2))/(sin(x/2))+(0.02379747526246cos(2x

In [20]:
avg = sum / (2**n+1)**2

minval, avg, maxval

# cook
# (0.14966, 0.49225, 0.97370)

# show
# (0.26886, 0.53202, 0.99996)

# had
# (0.00394, 0.61351, 0.99522)

# ideal
# 0.25 | 0.125 | 0.5
# (2.25e-32, 0.50141, 0.99959)

(2.25887530544004e-32, 0.501714048686793, 0.999588653828495)

## SXA

In [26]:
t0 = symbols('\\theta_0', real=True)
t1 = symbols('\\theta_1', real=True)
t2 = symbols('\\theta_2', real=True)
t3 = symbols('\\theta_3', real=True)
pi = symbols('\\pi', real=True)

In [27]:
phi0 = idx_to_vect("00")

phi1 = tfv(mtp2([H, H]), phi0)

phi2 = tfv(cgate2(rz(t0), 0, 1), phi1)

phi3 = tfv(cgate2(rz(t1), 1, 0), phi2)

phi4 = tfv(mtp2([H, H]), phi3)

phi5 = tfv(cgate2(rz(t0), 0, 1), phi4)

phi6 = tfv(cgate2(rz(t1), 1, 0), phi5)

In [28]:
# derive expression for <00|phi6>

raw00 = braket(idx_to_vect("00"), phi6).expand(trig=True).expand()

braket00 = simplify(raw00, maxn=17, chop=True)
temp = braket00
for a in preorder_traversal(temp):
    if isinstance(a, Float):
        braket00 = braket00.subs(a, round(a, 15))

display(braket00)

      ⅈ⋅\theta₀   ⅈ⋅\theta₁                -ⅈ⋅\theta₁          -ⅈ⋅\theta₀ 
      ───────── + ─────────                ───────────         ───────────
          2           2                         2                   2     
0.25⋅ℯ                      + 0.25 + 0.25⋅ℯ            + 0.25⋅ℯ           

In [29]:
# calculate P(|00>) = |<00|phi6>|

p00 = simplify(re(braket00)*re(braket00)+im(braket00)*im(braket00))

display(p00)

                                                             2                
       ⎛   ⎛\theta₀⎞      ⎛\theta₁⎞      ⎛\theta₀   \theta₁⎞⎞           ⎛   ⎛\
0.0625⋅⎜sin⎜───────⎟ + sin⎜───────⎟ - sin⎜─────── + ───────⎟⎟  + 0.0625⋅⎜cos⎜─
       ⎝   ⎝   2   ⎠      ⎝   2   ⎠      ⎝   2         2   ⎠⎠           ⎝   ⎝ 

                                                    2
theta₀⎞      ⎛\theta₁⎞      ⎛\theta₀   \theta₁⎞    ⎞ 
──────⎟ + cos⎜───────⎟ + cos⎜─────── + ───────⎟ + 1⎟ 
  2   ⎠      ⎝   2   ⎠      ⎝   2         2   ⎠    ⎠ 

In [30]:
# derive expression for <01|psi10>

raw01 = braket(idx_to_vect("01"), phi6).expand(trig=True).expand()

braket01 = simplify(raw01, maxn=17, chop=True)
temp = braket01
for a in preorder_traversal(temp):
    if isinstance(a, Float):
        braket01 = braket01.subs(a, round(a, 15))

display(braket01)

        ⅈ⋅\theta₀           ⅈ⋅\theta₀   ⅈ⋅\theta₁                            -
        ─────────         - ───────── - ─────────                            ─
            2                   2           2             -ⅈ⋅\theta₁          
- 0.25⋅ℯ          + 0.25⋅ℯ                        - 0.25⋅ℯ           + 0.25⋅ℯ 

ⅈ⋅\theta₁ 
──────────
    2     
          

In [31]:
# calculate P(|01>) = |<01|phi6>|

p01 = simplify(re(braket01)*re(braket01)+im(braket01)*im(braket01))

display(p01)

          ⎛\theta₀⎞    ⎛\theta₁⎞            ⎛\theta₀⎞            ⎛\theta₁⎞    
- 0.25⋅cos⎜───────⎟⋅cos⎜───────⎟ + 0.125⋅cos⎜───────⎟ - 0.125⋅cos⎜───────⎟ + 0
          ⎝   2   ⎠    ⎝   2   ⎠            ⎝   2   ⎠            ⎝   2   ⎠    

        ⎛\theta₀          ⎞            ⎛          \theta₁⎞       
.125⋅cos⎜─────── + \theta₁⎟ - 0.125⋅cos⎜\theta₀ + ───────⎟ + 0.25
        ⎝   2             ⎠            ⎝             2   ⎠       

In [32]:
p0_ = simplify(p00+p01)

display(p0_)

        ⎛\theta₀⎞           ⎛\theta₀          ⎞      
0.25⋅cos⎜───────⎟ + 0.25⋅cos⎜─────── + \theta₁⎟ + 0.5
        ⎝   2   ⎠           ⎝   2             ⎠      

In [33]:
# calculate P(O=|0>) = P(|00>)/( P(|00>)+P(|01>) )

f0 = simplify(p00/p0_)

display(f0)

                                                             2                
       ⎛   ⎛\theta₀⎞      ⎛\theta₁⎞      ⎛\theta₀   \theta₁⎞⎞           ⎛   ⎛\
0.0625⋅⎜sin⎜───────⎟ + sin⎜───────⎟ - sin⎜─────── + ───────⎟⎟  + 0.0625⋅⎜cos⎜─
       ⎝   ⎝   2   ⎠      ⎝   2   ⎠      ⎝   2         2   ⎠⎠           ⎝   ⎝ 
──────────────────────────────────────────────────────────────────────────────
                                               ⎛\theta₀⎞           ⎛\theta₀   
                                       0.25⋅cos⎜───────⎟ + 0.25⋅cos⎜─────── + 
                                               ⎝   2   ⎠           ⎝   2      

                                                    2
theta₀⎞      ⎛\theta₁⎞      ⎛\theta₀   \theta₁⎞    ⎞ 
──────⎟ + cos⎜───────⎟ + cos⎜─────── + ───────⎟ + 1⎟ 
  2   ⎠      ⎝   2   ⎠      ⎝   2         2   ⎠    ⎠ 
─────────────────────────────────────────────────────
       ⎞                                             
\theta₁⎟ + 0.5                             

In [34]:
simplified_f0 = ((cos(t1/4)+cos(3*t1/4))*cos(t1/4+t2/2))/(cos(t1/2)+cos(t1/2+t2)+2)+1/2

display(simplified_f0)

⎛   ⎛\theta₁⎞      ⎛3⋅\theta₁⎞⎞    ⎛\theta₁   \theta₂⎞      
⎜cos⎜───────⎟ + cos⎜─────────⎟⎟⋅cos⎜─────── + ───────⎟      
⎝   ⎝   4   ⎠      ⎝    4    ⎠⎠    ⎝   4         2   ⎠      
────────────────────────────────────────────────────── + 0.5
         ⎛\theta₁⎞      ⎛\theta₁          ⎞                 
      cos⎜───────⎟ + cos⎜─────── + \theta₂⎟ + 2             
         ⎝   2   ⎠      ⎝   2             ⎠                 

In [56]:
tau = 2*3.141592653589793

adj_0 = tau*0
adj_1 = tau*0

In [57]:
f0.subs([(t0, adj_0), (t1, adj_1)])

1.00000000000000

In [38]:
simplified_f0.subs([(t1, adj_0), (t2, adj_1)])

0

# Rx-CX

## SVAO

In [4]:
t0 = symbols('\\theta_0', real=True)
t1 = symbols('\\theta_1', real=True)
t2 = symbols('\\theta_2', real=True)
t3 = symbols('\\theta_3', real=True)
t4 = symbols('\\theta_4', real=True)
pi = symbols('\\pi', real=True)

In [5]:
# |psi0> = |0000>
psi0 = idx_to_vect("0000")

# |psi1> = Rx(t0)*Rx(t1)*I*Rx(t2) |psi0>
psi1 = tfv(mtp2([rx(t0), rx(t1), id(1), rx(t2)]), psi0)

# |psi2> = CNOT[0, 3] |psi1>
psi2 = tfv(cgate2(x(), 0, 3), psi1)

# |psi3> = CNOT[3, 1] |psi2>
psi3 = tfv(cgate2(x(), 3, 1), psi2)

# |psi4> = CNOT[1, 0]*I^2 |psi3>
psi4 = tfv(mtp2([cgate2(x(), 1, 0), id(1), id(1)]), psi3)

# |psi5> = Rx(t0)*Rx(t1)*I*Rx(t2) |psi4>
psi5 = tfv(mtp2([rx(t0), rx(t1), id(1), rx(t2)]), psi4)

# |psi6> = CNOT[0, 1]*I^2 |psi5>
psi6 = tfv(mtp2([cgate2(x(), 0, 1), id(1), id(1)]), psi5)

# |psi7> = CNOT[1, 3] |psi6>
psi7 = tfv(cgate2(x(), 1, 3), psi6)

# |psi8> = CNOT[3, 0] |psi7>
psi8 = tfv(cgate2(x(), 3, 0), psi7)

# |psi9> = CNOT[2, 3] |psi8>
psi9 = tfv(cgate2(x(), 2, 3), psi8)

# |psi10> = CNOT[3, 2] |psi9>
psi10 = tfv(cgate2(x(), 3, 2), psi9)

# |psi11> = I^2*Rx(t4)*Rx(t3) |psi10>
psi11 = tfv(mtp2([id(1), id(1), rx(t4), rx(t3)]), psi10)

# |psi12> = CNOT[2, 3] |psi11>
psi12 = tfv(cgate2(x(), 2, 3), psi11)

# |psi13> = CNOT[3, 2] |psi12>
psi13 = tfv(cgate2(x(), 3, 2), psi12)

# |psi14> = I^2*Rx(t4)*Rx(t3) |psi13>
psi14 = tfv(mtp2([id(1), id(1), rx(t4), rx(t3)]), psi13)

In [6]:
# for e in psi14:
#     print("---")
#     display(e)

In [7]:
# # derive expression for <0000|psi10>

# raw0000 = braket(idx_to_vect("0000"), psi14).expand(trig=True).expand()

# braket0000 = simplify(raw0000, maxn=17, chop=True)
# temp = braket0000
# for a in preorder_traversal(temp):
#     if isinstance(a, Float):
#         braket0000 = braket0000.subs(a, round(a, 15))

# display(braket0000)

In [8]:
# # calculate P(|0000>) = |<0000|psi10>|

# p0000 = simplify(re(braket0000)*re(braket0000)+im(braket0000)*im(braket0000))

# display(p0000)

In [9]:
p0000 = ((-sin(t1)-sin(t2)-sin(t4)-sin(t0+t3)+sin(t1-t2)-sin(t1+t4)-sin(t2+t4)-sin(-t0+t1+t3)+sin(-t0+t3+t4)-sin(t0-t2+t3)-sin(-t1+t2+t4)+sin(-t0+t2+t3+t4)+sin(t0-t1+t3+t4)+sin(t0+t1+t2-t3)+sin(t0+t1+t2+t3+t4))**2)/256+((+cos(t1)+cos(t2)+cos(t4)+cos(t0+t3)+cos(t1-t2)+cos(t1+t4)+cos(t2+t4)+cos(-t0+t1+t3)+cos(-t0+t3+t4)+cos(t0-t2+t3)+cos(-t1+t2+t4)+cos(-t0+t2+t3+t4)+cos(t0-t1+t3+t4)+cos(t0+t1+t2-t3)+cos(t0+t1+t2+t3+t4)+1)**2)/256

display(p0000)

                                                                              
(-sin(\theta₁) - sin(\theta₂) - sin(\theta₄) - sin(\theta₀ + \theta₃) + sin(\t
──────────────────────────────────────────────────────────────────────────────
                                                                              

                                                                              
heta₁ - \theta₂) - sin(\theta₁ + \theta₄) - sin(\theta₂ + \theta₄) - sin(-\the
──────────────────────────────────────────────────────────────────────────────
                                                                              

                                                                              
ta₀ + \theta₁ + \theta₃) + sin(-\theta₀ + \theta₃ + \theta₄) - sin(\theta₀ - \
──────────────────────────────────────────────────────────────────────────────
                                                                              

                                                 

In [10]:
# # derive expression for <0100|psi10>

# raw0100 = braket(idx_to_vect("0100"), psi14).expand(trig=True).expand()

# braket0100 = simplify(raw0100, maxn=17, chop=True)
# temp = braket0100
# for a in preorder_traversal(temp):
#     if isinstance(a, Float):
#         braket0100 = braket0100.subs(a, round(a, 15))

# display(braket0100)

In [11]:
# # calculate P(|0100>) = |<0100|psi10>|

# p0100 = simplify(re(braket0100)*re(braket0100)+im(braket0100)*im(braket0100))

# display(p0100)

In [12]:
p0100 = ((+sin(t1)-sin(t2)-sin(t4)+sin(t0+t3)-sin(t1-t2)+sin(t1+t4)-sin(t2+t4)-sin(-t0+t1+t3)-sin(-t0+t3+t4)+sin(t0-t2+t3)+sin(-t1+t2+t4)-sin(-t0+t2+t3+t4)+sin(t0-t1+t3+t4)+sin(t0+t1+t2-t3)+sin(t0+t1+t2+t3+t4))**2)/256+((-cos(t1)+cos(t2)+cos(t4)-cos(t0+t3)-cos(t1-t2)-cos(t1+t4)+cos(t2+t4)+cos(-t0+t1+t3)-cos(-t0+t3+t4)-cos(t0-t2+t3)-cos(-t1+t2+t4)-cos(-t0+t2+t3+t4)+cos(t0-t1+t3+t4)+cos(t0+t1+t2-t3)+cos(t0+t1+t2+t3+t4)+1)**2)/256

display(p0100)

                                                                              
(sin(\theta₁) - sin(\theta₂) - sin(\theta₄) + sin(\theta₀ + \theta₃) - sin(\th
──────────────────────────────────────────────────────────────────────────────
                                                                              

                                                                              
eta₁ - \theta₂) + sin(\theta₁ + \theta₄) - sin(\theta₂ + \theta₄) - sin(-\thet
──────────────────────────────────────────────────────────────────────────────
                                                                              

                                                                              
a₀ + \theta₁ + \theta₃) - sin(-\theta₀ + \theta₃ + \theta₄) + sin(\theta₀ - \t
──────────────────────────────────────────────────────────────────────────────
                                                                              

                                                 

In [13]:
p0_denom = simplify(p0000+p0100)

display(p0_denom)

cos(\theta₂)   cos(\theta₄)   cos(2⋅\theta₁ - \theta₂)   cos(2⋅\theta₁ + \thet
──────────── + ──────────── + ──────────────────────── + ─────────────────────
     16             16                   32                         32        

a₂)   cos(\theta₂ - \theta₄)   cos(\theta₂ + \theta₄)   cos(2⋅\theta₃ + \theta
─── + ────────────────────── + ────────────────────── + ──────────────────────
                64                       64                        16         

₄)   cos(-\theta₀ + \theta₁ + \theta₃)   cos(\theta₀ - \theta₁ + \theta₃)   co
── + ───────────────────────────────── + ──────────────────────────────── + ──
                     64                                 64                    

s(-2⋅\theta₁ + \theta₂ + \theta₄)   cos(2⋅\theta₁ - \theta₂ + \theta₄)   cos(-
───────────────────────────────── + ────────────────────────────────── + ─────
               64                                   64                        

\theta₂ + 2⋅\theta₃ + \theta₄)   cos(\theta₂ + 2

In [14]:
# calculate P(O=|0>) = P(|0000>)/( P(|0000>)+P(|0100>) )

p0 = (256*p0000)/(256*p0_denom)

display(p0)

                                                                              
                                                                              
──────────────────────────────────────────────────────────────────────────────
16⋅cos(\theta₂) + 16⋅cos(\theta₄) + 8⋅cos(2⋅\theta₁ - \theta₂) + 8⋅cos(2⋅\thet

                                                                              
                                                                              
──────────────────────────────────────────────────────────────────────────────
a₁ + \theta₂) + 4⋅cos(\theta₂ - \theta₄) + 4⋅cos(\theta₂ + \theta₄) + 16⋅cos(2

                                                                              
                                                                              
──────────────────────────────────────────────────────────────────────────────
⋅\theta₃ + \theta₄) + 4⋅cos(-\theta₀ + \theta₁ + \theta₃) + 4⋅cos(\theta₀ - \t

                                                 

In [15]:
# p0_00 = (4*cos(2*t1+t2)+4*cos(2*t1-t2)+2*cos(2*t3+t4)-4*cos(t0+t1-2*t2-t3-2*t4)-4*cos(t0+t1-t2-t3-2*t4)-4*cos(t0-t1-t2-t3-2*t4)-4*cos(t0-t1-t2-t3-t4)-4*cos(t0-t1-t3)-4*cos(t0-t1-t3-2*t4)-4*cos(t0-t1-t3-t4)+4*cos(t2)-4*cos(t2-2*t3-t4)+8*cos(t4)+4*cos(2*t1-t2+t4)+4*cos(2*t1+t2+2*t3+t4)+4*cos(t0-t1+t2+t3+2*t4)+4*cos(t0-t1+t2+t3+t4)+4*cos(t0-t1+t3+2*t4)+4*cos(t0-t1+t3+t4)+4*cos(t0+t1-2*t2+t3)+4*cos(t0+t1-t2+t3)+4*cos(t0+t1+2*t2-t3)+4*cos(t0+t1+2*t2-t3+t4)+4*cos(t0+t1+2*t2+t3+2*t4)+4*cos(t0+t1+2*t2+t3+t4)+4*cos(t0+t1+t2-t3)+4*cos(t0+t1+t2-t3+t4)+4*cos(t0+t1+t2+t3+2*t4)+4*cos(t0+t1+t2+t3+t4)+4*cos(t2+2*t3+t4)+4*cos(t2+t4))/256

# display(p0_00)

In [16]:
# p0 = simplify(p0000/p0_00)

# display(p0)

In [69]:
cook_0 = 1.204406074425311*3.141592653589793*2
cook_1 = 0.4647590541304515*3.141592653589793*2
cook_2 = 0.17828484324013236*3.141592653589793*2
unappet_0 = 0.9555832350160824*3.141592653589793*2
unappet_1 = 1.208010784363145*3.141592653589793*2

had_0 = -0.0035216290712991563*3.141592653589793*2
had_1 = 0.24421563296764226*3.141592653589793*2
had_2 = 0.671499679122132*3.141592653589793*2
yummy_0 = 0.45530682119641414*3.141592653589793*2
yummy_1 = 0.44391952311142013*3.141592653589793*2

had_0 = 0.0390380764748437*3.141592653589793*2
had_1 = 0.18034231033519194*3.141592653589793*2
had_2 = 0.6123173018475798*3.141592653589793*2
yummy_0 = 0.4317647167286927*3.141592653589793*2
yummy_1 = 0.2124684997913851*3.141592653589793*2

had_0 = 0.5943000301996968*3.141592653589793*2
had_1 = 0.33791122550713326*3.141592653589793*2
had_2 = 0.39161900052816123*3.141592653589793*2
yummy_0 = 0.931017315981155*3.141592653589793*2
yummy_1 = 0.040510711188434634*3.141592653589793*2

In [71]:
p0.subs([(t0, had_0), (t1, had_1), (t2, had_2)])

                                                                              
                                                                              
──────────────────────────────────────────────────────────────────────────────
16⋅cos(\theta₄) + 4⋅cos(\theta₃ - 9.07301377168931) + 4⋅cos(\theta₃ - 6.316198

                                                                              
                                                                              
──────────────────────────────────────────────────────────────────────────────
52551293) + 4⋅cos(\theta₃ - 5.22571069738875) + 4⋅cos(\theta₃ - 4.735143760334

                                                                              
                                                                              
──────────────────────────────────────────────────────────────────────────────
17) + 4⋅cos(\theta₃ - 2.95946238826695) + 4⋅cos(\theta₃ - 2.46889545121237) + 

                                                 

In [19]:
num = -2*cos(2*t1+t2)-2*cos(2*t1+t2-2*t3-t4)-2*cos(2*t1-t2)-2*cos(2*t3+t4)-4*cos(t0+t1-2*t2-t3-2*t4)-4*cos(t0+t1-t2-t3-2*t4)-4*cos(t0-t1-t2-t3-2*t4)-4*cos(t0-t1-t2-t3-t4)-4*cos(t0-t1-t3)-4*cos(t0-t1-t3-2*t4)-4*cos(t0-t1-t3-t4)+4*cos(t2)-4*cos(t2-2*t3-t4)+8*cos(t4)+4*cos(2*t1-t2+t4)+4*cos(2*t1+t2+2*t3+t4)+4*cos(t0-t1+t2+t3+2*t4)+4*cos(t0-t1+t2+t3+t4)+4*cos(t0-t1+t3+2*t4)+4*cos(t0-t1+t3+t4)+16

(num).subs([(t0, cook_0), (t1, cook_1), (t2, cook_2), (t3, unappet_0), (t4, unappet_1)])

24.8340089932729

In [20]:
latex = "\frac{\left(- \sin{\left(\theta_{4} \right)} - \sin{\left(\theta_{3} - 11.6078705185373 \right)} - \sin{\left(\theta_{3} - 4.64733929041582 \right)} - \sin{\left(\theta_{3} + 6.44730984316774 \right)} - \sin{\left(\theta_{3} + 7.56750655070696 \right)} - \sin{\left(\theta_{4} - 1.79997055275192 \right)} - \sin{\left(\theta_{4} + 1.12019670753922 \right)} - \sin{\left(\theta_{4} + 2.92016726029113 \right)} + \sin{\left(\theta_{3} + \theta_{4} - 7.56750655070696 \right)} + \sin{\left(\theta_{3} + \theta_{4} - 6.44730984316774 \right)} + \sin{\left(\theta_{3} + \theta_{4} + 4.64733929041582 \right)} + \sin{\left(\theta_{3} + \theta_{4} + 11.6078705185373 \right)} - 0.145952244451984\right)^{2} + \left(\cos{\left(\theta_{4} \right)} + \cos{\left(\theta_{3} - 11.6078705185373 \right)} + \cos{\left(\theta_{3} - 4.64733929041582 \right)} + \cos{\left(\theta_{3} + 6.44730984316774 \right)} + \cos{\left(\theta_{3} + 7.56750655070696 \right)} + \cos{\left(\theta_{4} - 1.79997055275192 \right)} + \cos{\left(\theta_{4} + 1.12019670753922 \right)} + \cos{\left(\theta_{4} + 2.92016726029113 \right)} + \cos{\left(\theta_{3} + \theta_{4} - 7.56750655070696 \right)} + \cos{\left(\theta_{3} + \theta_{4} - 6.44730984316774 \right)} + \cos{\left(\theta_{3} + \theta_{4} + 4.64733929041582 \right)} + \cos{\left(\theta_{3} + \theta_{4} + 11.6078705185373 \right)} + 0.232746568838004\right)^{2}}{16 \cos{\left(\theta_{4} \right)} + 4 \cos{\left(\theta_{3} - 12.7280672260765 \right)} + 4 \cos{\left(\theta_{3} - 11.6078705185373 \right)} + 4 \cos{\left(\theta_{3} - 5.76753599795504 \right)} + 4 \cos{\left(\theta_{3} - 4.64733929041582 \right)} + 4 \cos{\left(\theta_{3} + 3.52714258287661 \right)} + 4 \cos{\left(\theta_{3} + 4.64733929041582 \right)} + 4 \cos{\left(\theta_{3} + 8.24728039591966 \right)} + 4 \cos{\left(\theta_{3} + 9.36747710345888 \right)} + 16 \cos{\left(2 \theta_{3} + \theta_{4} \right)} + 4 \cos{\left(\theta_{4} - 4.72013781304305 \right)} + 4 \cos{\left(\theta_{4} - 1.12019670753922 \right)} + 4 \cos{\left(\theta_{4} + 1.12019670753922 \right)} + 4 \cos{\left(\theta_{4} + 4.72013781304305 \right)} + 4 \cos{\left(- \theta_{3} + \theta_{4} + 4.64733929041582 \right)} + 4 \cos{\left(- \theta_{3} + \theta_{4} + 5.76753599795504 \right)} + 4 \cos{\left(- \theta_{3} + \theta_{4} + 11.6078705185373 \right)} + 4 \cos{\left(- \theta_{3} + \theta_{4} + 12.7280672260765 \right)} + 4 \cos{\left(\theta_{3} - \theta_{4} + 3.52714258287661 \right)} + 4 \cos{\left(\theta_{3} - \theta_{4} + 4.64733929041582 \right)} + 4 \cos{\left(\theta_{3} - \theta_{4} + 8.24728039591966 \right)} + 4 \cos{\left(\theta_{3} - \theta_{4} + 9.36747710345888 \right)} + 4 \cos{\left(\theta_{3} + \theta_{4} - 9.36747710345888 \right)} + 4 \cos{\left(\theta_{3} + \theta_{4} - 8.24728039591966 \right)} + 4 \cos{\left(\theta_{3} + \theta_{4} - 4.64733929041582 \right)} + 4 \cos{\left(\theta_{3} + \theta_{4} - 3.52714258287661 \right)} + 4 \cos{\left(\theta_{3} + \theta_{4} + 4.64733929041582 \right)} + 4 \cos{\left(\theta_{3} + \theta_{4} + 5.76753599795504 \right)} + 4 \cos{\left(\theta_{3} + \theta_{4} + 11.6078705185373 \right)} + 4 \cos{\left(\theta_{3} + \theta_{4} + 12.7280672260765 \right)} + 4 \cos{\left(\theta_{3} + 2 \theta_{4} - 9.36747710345888 \right)} + 4 \cos{\left(\theta_{3} + 2 \theta_{4} - 8.24728039591966 \right)} + 4 \cos{\left(\theta_{3} + 2 \theta_{4} - 4.64733929041582 \right)} + 4 \cos{\left(\theta_{3} + 2 \theta_{4} - 3.52714258287661 \right)} + 4 \cos{\left(\theta_{3} + 2 \theta_{4} + 4.64733929041582 \right)} + 4 \cos{\left(\theta_{3} + 2 \theta_{4} + 5.76753599795504 \right)} + 4 \cos{\left(\theta_{3} + 2 \theta_{4} + 11.6078705185373 \right)} + 4 \cos{\left(\theta_{3} + 2 \theta_{4} + 12.7280672260765 \right)} + 4 \cos{\left(2 \theta_{3} + \theta_{4} - 6.96053122812148 \right)} + 4 \cos{\left(2 \theta_{3} + \theta_{4} - 1.12019670753922 \right)} + 4 \cos{\left(2 \theta_{3} + \theta_{4} + 1.12019670753922 \right)} + 4 \cos{\left(2 \theta_{3} + \theta_{4} + 6.96053122812148 \right)} + 45.2639868980053}"

In [72]:
latex = "\frac{\left(- \sin{\left(\theta_{4} \right)} - \sin{\left(\theta_{3} - 5.22571069738875 \right)} - \sin{\left(\theta_{3} - 3.60201960577327 \right)} - \sin{\left(\theta_{3} + 0.245283468527291 \right)} - \sin{\left(\theta_{3} + 0.887840686033608 \right)} - \sin{\left(\theta_{4} + 1.1331241545609 \right)} - \sin{\left(\theta_{4} + 2.71417891973966 \right)} - \sin{\left(\theta_{4} + 3.84730307430056 \right)} + \sin{\left(\theta_{3} + \theta_{4} - 0.887840686033608 \right)} + \sin{\left(\theta_{3} + \theta_{4} - 0.245283468527291 \right)} + \sin{\left(\theta_{3} + \theta_{4} + 3.60201960577327 \right)} + \sin{\left(\theta_{3} + \theta_{4} + 5.22571069738875 \right)} - 0.671684577657384\right)^{2} + 1.0 \left(\cos{\left(\theta_{4} \right)} + \cos{\left(\theta_{3} - 5.22571069738875 \right)} + \cos{\left(\theta_{3} - 3.60201960577327 \right)} + \cos{\left(\theta_{3} + 0.245283468527291 \right)} + \cos{\left(\theta_{3} + 0.887840686033608 \right)} + \cos{\left(\theta_{4} + 1.1331241545609 \right)} + \cos{\left(\theta_{4} + 2.71417891973966 \right)} + \cos{\left(\theta_{4} + 3.84730307430056 \right)} + \cos{\left(\theta_{3} + \theta_{4} - 0.887840686033608 \right)} + \cos{\left(\theta_{3} + \theta_{4} - 0.245283468527291 \right)} + \cos{\left(\theta_{3} + \theta_{4} + 3.60201960577327 \right)} + \cos{\left(\theta_{3} + \theta_{4} + 5.22571069738875 \right)} - 0.247359623453474\right)^{2}}{16 \cos{\left(\theta_{4} \right)} + 4 \cos{\left(\theta_{3} - 9.07301377168931 \right)} + 4 \cos{\left(\theta_{3} - 6.31619852551293 \right)} + 4 \cos{\left(\theta_{3} - 5.22571069738875 \right)} + 4 \cos{\left(\theta_{3} - 4.73514376033417 \right)} + 4 \cos{\left(\theta_{3} - 2.95946238826695 \right)} + 4 \cos{\left(\theta_{3} - 2.46889545121237 \right)} + 4 \cos{\left(\theta_{3} - 0.887840686033608 \right)} + 4 \cos{\left(\theta_{3} + 0.887840686033608 \right)} + 16 \cos{\left(2 \theta_{3} + \theta_{4} \right)} + 4 \cos{\left(\theta_{4} - 3.84730307430056 \right)} + 4 \cos{\left(\theta_{4} - 1.58105476517876 \right)} + 4 \cos{\left(\theta_{4} + 1.58105476517876 \right)} + 4 \cos{\left(\theta_{4} + 3.84730307430056 \right)} + 4 \cos{\left(- \theta_{3} + \theta_{4} + 0.887840686033608 \right)} + 4 \cos{\left(- \theta_{3} + \theta_{4} + 2.46889545121237 \right)} + 4 \cos{\left(- \theta_{3} + \theta_{4} + 2.95946238826695 \right)} + 4 \cos{\left(- \theta_{3} + \theta_{4} + 4.73514376033417 \right)} + 4 \cos{\left(- \theta_{3} + \theta_{4} + 5.22571069738875 \right)} + 4 \cos{\left(- \theta_{3} + \theta_{4} + 6.31619852551293 \right)} + 4 \cos{\left(- \theta_{3} + \theta_{4} + 9.07301377168931 \right)} + 4 \cos{\left(\theta_{3} - \theta_{4} + 0.887840686033608 \right)} + 4 \cos{\left(\theta_{3} + \theta_{4} - 0.887840686033608 \right)} + 4 \cos{\left(\theta_{3} + \theta_{4} + 0.887840686033608 \right)} + 4 \cos{\left(\theta_{3} + \theta_{4} + 2.46889545121237 \right)} + 4 \cos{\left(\theta_{3} + \theta_{4} + 2.95946238826695 \right)} + 4 \cos{\left(\theta_{3} + \theta_{4} + 4.73514376033417 \right)} + 4 \cos{\left(\theta_{3} + \theta_{4} + 5.22571069738875 \right)} + 4 \cos{\left(\theta_{3} + \theta_{4} + 6.31619852551293 \right)} + 4 \cos{\left(\theta_{3} + \theta_{4} + 9.07301377168931 \right)} + 4 \cos{\left(\theta_{3} + 2 \theta_{4} - 0.887840686033608 \right)} + 4 \cos{\left(\theta_{3} + 2 \theta_{4} + 0.887840686033608 \right)} + 4 \cos{\left(\theta_{3} + 2 \theta_{4} + 2.46889545121237 \right)} + 4 \cos{\left(\theta_{3} + 2 \theta_{4} + 2.95946238826695 \right)} + 4 \cos{\left(\theta_{3} + 2 \theta_{4} + 4.73514376033417 \right)} + 4 \cos{\left(\theta_{3} + 2 \theta_{4} + 5.22571069738875 \right)} + 4 \cos{\left(\theta_{3} + 2 \theta_{4} + 6.31619852551293 \right)} + 4 \cos{\left(\theta_{3} + 2 \theta_{4} + 9.07301377168931 \right)} + 4 \cos{\left(2 \theta_{3} + \theta_{4} - 6.11355138342236 \right)} + 4 \cos{\left(2 \theta_{3} + \theta_{4} - 3.84730307430056 \right)} + 4 \cos{\left(2 \theta_{3} + \theta_{4} + 3.84730307430056 \right)} + 4 \cos{\left(2 \theta_{3} + \theta_{4} + 6.11355138342236 \right)} + 27.6246912810956}"

In [73]:
latex = latex.replace("\left", "").replace("\right", "")

latex = latex.replace("\f", "f")

latex = latex.replace("\theta_", "t").replace("\sin", "sin").replace("\cos", "cos")

for i in range(10):
    latex = latex.replace("t{"+str(i)+"}", "t"+str(i))

latex = latex.replace("}{", "}/{").replace("frac", "").replace("{(", "{").replace(")}", "}").replace("{", "(").replace("}", ")").replace(" ", "")

latex = latex.replace("t3", "x").replace("t4", "y")

latex = latex.replace("(2)", "2") #.replace("(x)", "x").replace("(y)", "y")

# latex = latex.replace("x", "2πx").replace("y", "2πy")




In [74]:
latex

'(-sin(y)-sin(x-5.22571069738875)-sin(x-3.60201960577327)-sin(x+0.245283468527291)-sin(x+0.887840686033608)-sin(y+1.1331241545609)-sin(y+2.71417891973966)-sin(y+3.84730307430056)+sin(x+y-0.887840686033608)+sin(x+y-0.245283468527291)+sin(x+y+3.60201960577327)+sin(x+y+5.22571069738875)-0.671684577657384)^2+1.0(cos(y)+cos(x-5.22571069738875)+cos(x-3.60201960577327)+cos(x+0.245283468527291)+cos(x+0.887840686033608)+cos(y+1.1331241545609)+cos(y+2.71417891973966)+cos(y+3.84730307430056)+cos(x+y-0.887840686033608)+cos(x+y-0.245283468527291)+cos(x+y+3.60201960577327)+cos(x+y+5.22571069738875)-0.247359623453474)^2)/(16cos(y)+4cos(x-9.07301377168931)+4cos(x-6.31619852551293)+4cos(x-5.22571069738875)+4cos(x-4.73514376033417)+4cos(x-2.95946238826695)+4cos(x-2.46889545121237)+4cos(x-0.887840686033608)+4cos(x+0.887840686033608)+16cos(2x+y)+4cos(y-3.84730307430056)+4cos(y-1.58105476517876)+4cos(y+1.58105476517876)+4cos(y+3.84730307430056)+4cos(-x+y+0.887840686033608)+4cos(-x+y+2.46889545121237)+4cos(

## SXA

In [83]:
t0 = symbols('\\theta_0', real=True)
t1 = symbols('\\theta_1', real=True)
pi = symbols('\\pi', real=True)

In [94]:
# |phi0> = |00>
phi0 = idx_to_vect("00")

# |phi1> = Rx(t0)*Rx(t1) |phi0>
phi1 = tfv(mtp2([rx(t0), rx(t1)]), phi0)

# |phi2> = CNOT[0, 1] |phi1>
phi2 = tfv(cgate2(x(), 0, 1), phi1)

# |phi3> = CNOT[1, 0] |phi2>
phi3 = tfv(cgate2(x(), 1, 0), phi2)

# |phi4> = Rx(t0)*Rx(t1) |phi3>
phi4 = tfv(mtp2([rx(t0), rx(t1)]), phi3)

# |phi5> = CNOT[0, 1] |phi4>
phi5 = tfv(cgate2(x(), 0, 1), phi4)

# |phi6> = CNOT[1, 0] |phi5>
phi6 = tfv(cgate2(x(), 1, 0), phi5)

In [95]:
# derive expression for <00|phi6>

raw00 = braket(idx_to_vect("00"), phi6).expand(trig=True).expand()

braket00 = simplify(raw00, maxn=17, chop=True)
temp = braket00
for a in preorder_traversal(temp):
    if isinstance(a, Float):
        braket00 = braket00.subs(a, round(a, 15))

display(braket00)

 ⅈ⋅\theta₀    ⅈ⋅\theta₁    ⅈ⋅(-\theta₀ - \theta₁)    
ℯ            ℯ            ℯ                         1
────────── + ────────── + ─────────────────────── + ─
    4            4                   4              4

In [96]:
# calculate P(|00>) = |<00|phi6>|

p00 = simplify(re(braket00)*re(braket00)+im(braket00)*im(braket00))

display(p00)

                                                      2                       
(sin(\theta₀) + sin(\theta₁) - sin(\theta₀ + \theta₁))    (cos(\theta₀) + cos(
─────────────────────────────────────────────────────── + ────────────────────
                           16                                                 

                                      2
\theta₁) + cos(\theta₀ + \theta₁) + 1) 
───────────────────────────────────────
         16                            

In [97]:
# derive expression for <01|psi10>

raw01 = braket(idx_to_vect("01"), phi6).expand(trig=True).expand()

braket01 = simplify(raw01, maxn=17, chop=True)
temp = braket01
for a in preorder_traversal(temp):
    if isinstance(a, Float):
        braket01 = braket01.subs(a, round(a, 15))

display(braket01)

   ⅈ⋅\theta₀    ⅈ⋅\theta₁        -ⅈ⋅(\theta₀ + \theta₁)
  ℯ            ℯ            1   ℯ                      
- ────────── + ────────── - ─ + ───────────────────────
      4            4        4              4           

In [98]:
# calculate P(|01>) = |<01|phi6>|

p01 = simplify(re(braket01)*re(braket01)+im(braket01)*im(braket01))

display(p01)

                                                      2                       
(sin(\theta₀) - sin(\theta₁) + sin(\theta₀ + \theta₁))    (-cos(\theta₀) + cos
─────────────────────────────────────────────────────── + ────────────────────
                           16                                                 

                                       2
(\theta₁) + cos(\theta₀ + \theta₁) - 1) 
────────────────────────────────────────
         16                             

In [99]:
p0_ = simplify(p00+p01)

display(p0_)

cos(\theta₀)   cos(\theta₀ + 2⋅\theta₁)   1
──────────── + ──────────────────────── + ─
     4                    4               2

In [100]:
# calculate P(O=|0>) = P(|00>)/( P(|00>)+P(|01>) )

f0 = simplify(p00/p0_)

display(f0)

                                                      2                       
(sin(\theta₀) + sin(\theta₁) - sin(\theta₀ + \theta₁))  + (cos(\theta₀) + cos(
──────────────────────────────────────────────────────────────────────────────
                                   4⋅(cos(\theta₀) + cos(\theta₀ + 2⋅\theta₁) 

                                      2
\theta₁) + cos(\theta₀ + \theta₁) + 1) 
───────────────────────────────────────
+ 2)                                   

In [109]:
slow_0 = 0.7436994864265065*3.141592653589796*2
slow_1 = 0.4019002667598174*3.141592653589796*2

In [110]:
f0.subs([(t0, slow_0), (t1, slow_1)])

0.508117328542717

In [123]:
num = 2*(cos(t1)+cos(t0+t1)+cos(t0-t1)+cos(2*t0+t1))

display(num)

2⋅cos(\theta₁) + 2⋅cos(\theta₀ - \theta₁) + 2⋅cos(\theta₀ + \theta₁) + 2⋅cos(2
⋅\theta₀ + \theta₁)

In [124]:
denom = (4*(cos(t0)+cos(t0+2*t1)+2))

display(denom)

4⋅cos(\theta₀) + 4⋅cos(\theta₀ + 2⋅\theta₁) + 8

In [125]:
num.subs([(t0, slow_0), (t1, slow_1)])

0.0326196492652886

In [126]:
denom.subs([(t0, slow_0), (t1, slow_1)])

4.01852026730583

In [138]:
(cos(t0)).subs([(t0, slow_0), (t1, slow_1)])

-0.0395769552259158

In [143]:
zero = 2*(cos(t1)+cos(t0+t1)+cos(t0-t1)+cos(2*t0+t1))/(4*(cos(t0)+cos(t0+2*t1)+2))

zero.subs([(t0, slow_0), (t1, slow_1)])

0.00811732854271707

In [155]:
(cos(t0)).subs([(t0, 0), (t1, slow_1)])

1

In [156]:
(cos(t1)).subs([(t0,0), (t1, slow_1)])

-0.815977158898529

In [157]:
(cos(t0+t1)).subs([(t0, 0), (t1, slow_1)])

-0.815977158898529

# Rx-CRz

## SVAO

In [92]:
t0 = symbols('\\theta_0', real=True)
t1 = symbols('\\theta_1', real=True)
t2 = symbols('\\theta_2', real=True)
t3 = symbols('\\theta_3', real=True)
t4 = symbols('\\theta_4', real=True)
t5 = symbols('\\theta_5', real=True)
t6 = symbols('\\theta_6', real=True)
t7 = symbols('\\theta_7', real=True)
t8 = symbols('\\theta_8', real=True)
t9 = symbols('\\theta_9', real=True)
pi = symbols('\\pi', real=True)

In [93]:
# |psi0> = |0000>
psi0 = idx_to_vect("0000")

# |psi1> = rx(t0)*rx(t1)*I*rx(t2) |psi0>
psi1 = tfv(mtp2([rx(t0), rx(t1), id(1), rx(t2)]), psi0)

# |psi2> = CRz[0, 3](t3) |psi1>
psi2 = tfv(cgate2(rz(t3), 0, 3), psi1)

# |psi3> = CRz[3, 1](t4) |psi2>
psi3 = tfv(cgate2(rz(t4), 3, 1), psi2)

# |psi4> = CRz[1, 0](t5)*I^2 |psi3>
psi4 = tfv(mtp2([cgate2(rz(t5), 1, 0), id(1), id(1)]), psi3)

# |psi5> = rx(t0)*rx(t1)*I*rx(t2) |psi4>
psi5 = tfv(mtp2([rx(t0), rx(t1), id(1), rx(t2)]), psi4)

# |psi6> = CRz[0, 1](t3)*I^2 |psi5>
psi6 = tfv(mtp2([cgate2(rz(t3), 0, 1), id(1), id(1)]), psi5)

# |psi7> = CRz[1, 3](t4) |psi6>
psi7 = tfv(cgate2(rz(t4), 1, 3), psi6)

# |psi8> = CRz[3, 0](t5) |psi7>
psi8 = tfv(cgate2(rz(t5), 3, 0), psi7)

# |psi9> = CRz[2, 3](t9) |psi8>
psi9 = tfv(cgate2(rz(t9), 2, 3), psi8)

# |psi10> = CRz[3, 2](t8) |psi9>
psi10 = tfv(cgate2(rz(t8), 3, 2), psi9)

# |psi11> = I^2*rx(t7)*rx(t6) |psi10>
psi11 = tfv(mtp2([id(1), id(1), rx(t7), rx(t6)]), psi10)

# |psi12> = CRz[2, 3](t9) |psi11>
psi12 = tfv(cgate2(rz(t9), 2, 3), psi11)

# |psi13> = CRz[3, 2](t8) |psi12>
psi13 = tfv(cgate2(rz(t8), 3, 2), psi12)

# |psi14> = I^2*rx(t7)*rx(t6) |psi13>
psi14 = tfv(mtp2([id(1), id(1), rx(t7), rx(t6)]), psi13)


In [94]:
proj0000 = braket(idx_to_vect("0000"), psi14).expand(trig=True).expand()

p0000 = (re(proj0000)*re(proj0000)+im(proj0000)*im(proj0000))

proj0100 = braket(idx_to_vect("0100"), psi14).expand(trig=True).expand()

p0100 = (re(proj0100)*re(proj0100)+im(proj0100)*im(proj0100))

p0_denom = (p0000+p0100)

In [95]:
tau = 2*3.141592653589796

verb_0 = tau*1.9340185332827495
verb_1 = tau*0.7880610218763398
verb_2 = tau*2.2087825995359056
verb_3 = tau*0.9496007155089805
verb_4 = tau*1.660665072847281
verb_5 = tau*1.6037007145170072
adj_0 = tau*-1.0861949753754054
adj_1 = tau*1.0666182679966156
adj_2 = tau*-0.32709247539028496
adj_3 = tau*2.546949283251263

p0_num = p0000.subs([(t0, verb_0), (t1, verb_1), (t2, verb_2), (t3, verb_3), (t4, verb_4), (t5, verb_5), (t6, adj_0), (t7, adj_1), (t8, adj_2), (t9, adj_3)])

p0 = p0_num/p0_denom.subs([(t0, verb_0), (t1, verb_1), (t2, verb_2), (t3, verb_3), (t4, verb_4), (t5, verb_5), (t6, adj_0), (t7, adj_1), (t8, adj_2), (t9, adj_3)])

p0

0.418614155505566

In [96]:
p0 = p0/p0_denom

display(p0)

                                                                              
──────────────────────────────────────────────────────────────────────────────
                                                                              
⎛                                                                          ⎛ ⅈ
⎜                                                                          ⎜ ─
⎜     2⎛\theta₀⎞    2⎛\theta₁⎞    2⎛\theta₂⎞    2⎛\theta₆⎞    2⎛\theta₇⎞   ⎜  
⎜- sin ⎜───────⎟⋅sin ⎜───────⎟⋅sin ⎜───────⎟⋅sin ⎜───────⎟⋅sin ⎜───────⎟⋅im⎝ℯ 
⎝      ⎝   2   ⎠     ⎝   2   ⎠     ⎝   2   ⎠     ⎝   2   ⎠     ⎝   2   ⎠      

                                                                              
──────────────────────────────────────────────────────────────────────────────
                                                                              
⋅\theta₃  ⅈ⋅\theta₄  ⅈ⋅\theta₅  ⅈ⋅\theta₈  ⅈ⋅\theta₉⎞                         
────────  ─────────  ─────────  ─────────  ────────

In [98]:
s0 = simplify(p0.subs([(t0, verb_0), (t1, verb_1), (t2, verb_2), (t3, verb_3), (t4, verb_4), (t5, verb_5)]))

In [99]:
s0

                                                                              
──────────────────────────────────────────────────────────────────────────────
                                                                              
                    ⎛     ⎛\theta₈⎞      ⎛\theta₉⎞                            
0.00930674450091877⋅⎜- sin⎜───────⎟ - sin⎜───────⎟ - 0.00269431212566771⋅sin(\
                    ⎝     ⎝   2   ⎠      ⎝   2   ⎠                            

                                                                              
──────────────────────────────────────────────────────────────────────────────
                                                                              
                                         ⎛          \theta₈⎞                  
theta₆ - \theta₈) - 0.502694312125668⋅sin⎜\theta₆ - ───────⎟ + 0.4973056878743
                                         ⎝             2   ⎠                  

                                                  

In [106]:
latex = "\frac{0.418614155505566}{0.00930674450091877 \left(- \sin{\left(\frac{\theta_{8}}{2} \right)} - \sin{\left(\frac{\theta_{9}}{2} \right)} - 0.00269431212566771 \sin{\left(\theta_{6} - \theta_{8} \right)} - 0.502694312125668 \sin{\left(\theta_{6} - \frac{\theta_{8}}{2} \right)} + 0.497305687874332 \sin{\left(\theta_{6} + \frac{\theta_{8}}{2} \right)} - 0.00269431212566771 \sin{\left(\theta_{6} + \theta_{8} \right)} + 0.502694312125668 \sin{\left(\theta_{6} - \frac{\theta_{9}}{2} \right)} - 0.497305687874332 \sin{\left(\theta_{6} + \frac{\theta_{9}}{2} \right)} + 0.5 \sin{\left(\theta_{7} - \frac{\theta_{8}}{2} \right)} - 0.5 \sin{\left(\theta_{7} + \frac{\theta_{8}}{2} \right)} - 0.5 \sin{\left(\theta_{7} - \frac{\theta_{9}}{2} \right)} + 0.5 \sin{\left(\theta_{7} + \frac{\theta_{9}}{2} \right)} - \sin{\left(\frac{\theta_{8}}{2} + \frac{\theta_{9}}{2} \right)} + 0.251347156062834 \sin{\left(- \theta_{6} + \theta_{7} + \frac{\theta_{8}}{2} \right)} + 0.00134715606283385 \sin{\left(- \theta_{6} + \theta_{7} + \theta_{8} \right)} + 0.251347156062834 \sin{\left(- \theta_{6} + \theta_{7} + \frac{\theta_{9}}{2} \right)} + 0.497305687874332 \sin{\left(- \theta_{6} + \frac{\theta_{8}}{2} + \frac{\theta_{9}}{2} \right)} + 0.248652843937166 \sin{\left(\theta_{6} - \theta_{7} + \frac{\theta_{8}}{2} \right)} - 0.00134715606283385 \sin{\left(\theta_{6} - \theta_{7} + \theta_{8} \right)} + 0.248652843937166 \sin{\left(\theta_{6} - \theta_{7} + \frac{\theta_{9}}{2} \right)} - 0.00134715606283385 \sin{\left(\theta_{6} + \theta_{7} - \theta_{8} \right)} - 0.251347156062834 \sin{\left(\theta_{6} + \theta_{7} - \frac{\theta_{8}}{2} \right)} + 0.248652843937166 \sin{\left(\theta_{6} + \theta_{7} + \frac{\theta_{8}}{2} \right)} - 0.00134715606283385 \sin{\left(\theta_{6} + \theta_{7} + \theta_{8} \right)} - 0.251347156062834 \sin{\left(\theta_{6} + \theta_{7} - \frac{\theta_{9}}{2} \right)} + 0.248652843937166 \sin{\left(\theta_{6} + \theta_{7} + \frac{\theta_{9}}{2} \right)} + 0.502694312125668 \sin{\left(\theta_{6} + \frac{\theta_{8}}{2} + \frac{\theta_{9}}{2} \right)} + 0.5 \sin{\left(- \theta_{7} + \frac{\theta_{8}}{2} + \frac{\theta_{9}}{2} \right)} + 0.5 \sin{\left(\theta_{7} + \frac{\theta_{8}}{2} + \frac{\theta_{9}}{2} \right)} - 0.248652843937166 \sin{\left(- \theta_{6} + \theta_{7} + \frac{\theta_{8}}{2} + \frac{\theta_{9}}{2} \right)} - 0.251347156062834 \sin{\left(\theta_{6} - \theta_{7} + \frac{\theta_{8}}{2} + \frac{\theta_{9}}{2} \right)} + 0.248652843937166 \sin{\left(\theta_{6} + \theta_{7} - \frac{\theta_{8}}{2} - \frac{\theta_{9}}{2} \right)} - 0.251347156062834 \sin{\left(\theta_{6} + \theta_{7} + \frac{\theta_{8}}{2} + \frac{\theta_{9}}{2} \right)} - 0.846212986949131 \cos{\left(\theta_{6} \right)} - 0.846212986949131 \cos{\left(\theta_{7} \right)} + 0.846212986949131 \cos{\left(\frac{\theta_{8}}{2} \right)} + 0.846212986949131 \cos{\left(\frac{\theta_{9}}{2} \right)} - 0.423106493474565 \cos{\left(\theta_{6} - \theta_{7} \right)} - 0.423106493474565 \cos{\left(\theta_{6} + \theta_{7} \right)} + 0.29786835563659 \cos{\left(\theta_{6} - \theta_{8} \right)} - 0.125238137837975 \cos{\left(\theta_{6} - \frac{\theta_{8}}{2} \right)} - 0.720974849111156 \cos{\left(\theta_{6} + \frac{\theta_{8}}{2} \right)} - 0.29786835563659 \cos{\left(\theta_{6} + \theta_{8} \right)} + 0.720974849111156 \cos{\left(\theta_{6} - \frac{\theta_{9}}{2} \right)} + 0.125238137837975 \cos{\left(\theta_{6} + \frac{\theta_{9}}{2} \right)} + 0.423106493474566 \cos{\left(\theta_{7} - \frac{\theta_{8}}{2} \right)} + 0.423106493474566 \cos{\left(\theta_{7} + \frac{\theta_{8}}{2} \right)} - 0.423106493474566 \cos{\left(\theta_{7} - \frac{\theta_{9}}{2} \right)} - 0.423106493474566 \cos{\left(\theta_{7} + \frac{\theta_{9}}{2} \right)} - 0.846212986949131 \cos{\left(\frac{\theta_{8}}{2} + \frac{\theta_{9}}{2} \right)} - 0.0626190689189876 \cos{\left(- \theta_{6} + \theta_{7} + \frac{\theta_{8}}{2} \right)} + 0.148934177818295 \cos{\left(- \theta_{6} + \theta_{7} + \theta_{8} \right)} - 0.360487424555578 \cos{\left(- \theta_{6} + \theta_{7} + \frac{\theta_{9}}{2} \right)} + 0.125238137837975 \cos{\left(- \theta_{6} + \frac{\theta_{8}}{2} + \frac{\theta_{9}}{2} \right)} - 0.360487424555578 \cos{\left(\theta_{6} - \theta_{7} + \frac{\theta_{8}}{2} \right)} - 0.148934177818295 \cos{\left(\theta_{6} - \theta_{7} + \theta_{8} \right)} - 0.0626190689189876 \cos{\left(\theta_{6} - \theta_{7} + \frac{\theta_{9}}{2} \right)} + 0.148934177818295 \cos{\left(\theta_{6} + \theta_{7} - \theta_{8} \right)} - 0.0626190689189876 \cos{\left(\theta_{6} + \theta_{7} - \frac{\theta_{8}}{2} \right)} - 0.360487424555578 \cos{\left(\theta_{6} + \theta_{7} + \frac{\theta_{8}}{2} \right)} - 0.148934177818295 \cos{\left(\theta_{6} + \theta_{7} + \theta_{8} \right)} - 0.360487424555578 \cos{\left(\theta_{6} + \theta_{7} - \frac{\theta_{9}}{2} \right)} - 0.0626190689189876 \cos{\left(\theta_{6} + \theta_{7} + \frac{\theta_{9}}{2} \right)} + 0.720974849111156 \cos{\left(\theta_{6} + \frac{\theta_{8}}{2} + \frac{\theta_{9}}{2} \right)} + 0.423106493474565 \cos{\left(- \theta_{7} + \frac{\theta_{8}}{2} + \frac{\theta_{9}}{2} \right)} + 0.423106493474565 \cos{\left(\theta_{7} + \frac{\theta_{8}}{2} + \frac{\theta_{9}}{2} \right)} - 0.0626190689189876 \cos{\left(- \theta_{6} + \theta_{7} + \frac{\theta_{8}}{2} + \frac{\theta_{9}}{2} \right)} - 0.360487424555578 \cos{\left(\theta_{6} - \theta_{7} + \frac{\theta_{8}}{2} + \frac{\theta_{9}}{2} \right)} - 0.0626190689189876 \cos{\left(\theta_{6} + \theta_{7} - \frac{\theta_{8}}{2} - \frac{\theta_{9}}{2} \right)} - 0.360487424555578 \cos{\left(\theta_{6} + \theta_{7} + \frac{\theta_{8}}{2} + \frac{\theta_{9}}{2} \right)} - 0.846212986949131\right)^{2} + 0.00818328463224793 \left(- 0.479249714471875 \sin{\left(\frac{\theta_{8}}{2} \right)} - 0.479249714471875 \sin{\left(\frac{\theta_{9}}{2} \right)} - 0.760375142764063 \sin{\left(\theta_{6} - \theta_{8} \right)} - \sin{\left(\theta_{6} - \frac{\theta_{8}}{2} \right)} - 0.520750285528125 \sin{\left(\theta_{6} + \frac{\theta_{8}}{2} \right)} - 0.760375142764063 \sin{\left(\theta_{6} + \theta_{8} \right)} + \sin{\left(\theta_{6} - \frac{\theta_{9}}{2} \right)} + 0.520750285528125 \sin{\left(\theta_{6} + \frac{\theta_{9}}{2} \right)} + 0.239624857235937 \sin{\left(\theta_{7} - \frac{\theta_{8}}{2} \right)} - 0.239624857235937 \sin{\left(\theta_{7} + \frac{\theta_{8}}{2} \right)} - 0.239624857235937 \sin{\left(\theta_{7} - \frac{\theta_{9}}{2} \right)} + 0.239624857235937 \sin{\left(\theta_{7} + \frac{\theta_{9}}{2} \right)} - 0.479249714471875 \sin{\left(\frac{\theta_{8}}{2} + \frac{\theta_{9}}{2} \right)} + 0.5 \sin{\left(- \theta_{6} + \theta_{7} + \frac{\theta_{8}}{2} \right)} + 0.380187571382031 \sin{\left(- \theta_{6} + \theta_{7} + \theta_{8} \right)} + 0.5 \sin{\left(- \theta_{6} + \theta_{7} + \frac{\theta_{9}}{2} \right)} - 0.520750285528126 \sin{\left(- \theta_{6} + \frac{\theta_{8}}{2} + \frac{\theta_{9}}{2} \right)} - 0.260375142764063 \sin{\left(\theta_{6} - \theta_{7} + \frac{\theta_{8}}{2} \right)} - 0.380187571382031 \sin{\left(\theta_{6} - \theta_{7} + \theta_{8} \right)} - 0.260375142764063 \sin{\left(\theta_{6} - \theta_{7} + \frac{\theta_{9}}{2} \right)} - 0.380187571382031 \sin{\left(\theta_{6} + \theta_{7} - \theta_{8} \right)} - 0.5 \sin{\left(\theta_{6} + \theta_{7} - \frac{\theta_{8}}{2} \right)} - 0.260375142764063 \sin{\left(\theta_{6} + \theta_{7} + \frac{\theta_{8}}{2} \right)} - 0.380187571382031 \sin{\left(\theta_{6} + \theta_{7} + \theta_{8} \right)} - 0.5 \sin{\left(\theta_{6} + \theta_{7} - \frac{\theta_{9}}{2} \right)} - 0.260375142764063 \sin{\left(\theta_{6} + \theta_{7} + \frac{\theta_{9}}{2} \right)} + \sin{\left(\theta_{6} + \frac{\theta_{8}}{2} + \frac{\theta_{9}}{2} \right)} + 0.239624857235937 \sin{\left(- \theta_{7} + \frac{\theta_{8}}{2} + \frac{\theta_{9}}{2} \right)} + 0.239624857235937 \sin{\left(\theta_{7} + \frac{\theta_{8}}{2} + \frac{\theta_{9}}{2} \right)} + 0.260375142764063 \sin{\left(- \theta_{6} + \theta_{7} + \frac{\theta_{8}}{2} + \frac{\theta_{9}}{2} \right)} - 0.5 \sin{\left(\theta_{6} - \theta_{7} + \frac{\theta_{8}}{2} + \frac{\theta_{9}}{2} \right)} - 0.260375142764063 \sin{\left(\theta_{6} + \theta_{7} - \frac{\theta_{8}}{2} - \frac{\theta_{9}}{2} \right)} - 0.5 \sin{\left(\theta_{6} + \theta_{7} + \frac{\theta_{8}}{2} + \frac{\theta_{9}}{2} \right)} - 4.86561920156945 \cdot 10^{-5} \cos{\left(\theta_{6} \right)} - 4.86561920156945 \cdot 10^{-5} \cos{\left(\theta_{7} \right)} + 4.86561920156721 \cdot 10^{-5} \cos{\left(\frac{\theta_{8}}{2} \right)} + 4.86561920156721 \cdot 10^{-5} \cos{\left(\frac{\theta_{9}}{2} \right)} - 2.43280960078473 \cdot 10^{-5} \cos{\left(\theta_{6} - \theta_{7} \right)} - 2.43280960078473 \cdot 10^{-5} \cos{\left(\theta_{6} + \theta_{7} \right)} - 0.761377913238866 \cos{\left(\theta_{6} - \theta_{8} \right)} - 0.761402241334874 \cos{\left(\theta_{6} - \frac{\theta_{8}}{2} \right)} + 0.761353585142858 \cos{\left(\theta_{6} + \frac{\theta_{8}}{2} \right)} + 0.761377913238866 \cos{\left(\theta_{6} + \theta_{8} \right)} - 0.761353585142858 \cos{\left(\theta_{6} - \frac{\theta_{9}}{2} \right)} + 0.761402241334874 \cos{\left(\theta_{6} + \frac{\theta_{9}}{2} \right)} + 2.43280960078361 \cdot 10^{-5} \cos{\left(\theta_{7} - \frac{\theta_{8}}{2} \right)} + 2.43280960078361 \cdot 10^{-5} \cos{\left(\theta_{7} + \frac{\theta_{8}}{2} \right)} - 2.43280960078361 \cdot 10^{-5} \cos{\left(\theta_{7} - \frac{\theta_{9}}{2} \right)} - 2.43280960078361 \cdot 10^{-5} \cos{\left(\theta_{7} + \frac{\theta_{9}}{2} \right)} - 4.8656192015726 \cdot 10^{-5} \cos{\left(\frac{\theta_{8}}{2} + \frac{\theta_{9}}{2} \right)} - 0.380701120667437 \cos{\left(- \theta_{6} + \theta_{7} + \frac{\theta_{8}}{2} \right)} - 0.380688956619433 \cos{\left(- \theta_{6} + \theta_{7} + \theta_{8} \right)} + 0.380676792571429 \cos{\left(- \theta_{6} + \theta_{7} + \frac{\theta_{9}}{2} \right)} + 0.761402241334874 \cos{\left(- \theta_{6} + \frac{\theta_{8}}{2} + \frac{\theta_{9}}{2} \right)} + 0.380676792571429 \cos{\left(\theta_{6} - \theta_{7} + \frac{\theta_{8}}{2} \right)} + 0.380688956619433 \cos{\left(\theta_{6} - \theta_{7} + \theta_{8} \right)} - 0.380701120667437 \cos{\left(\theta_{6} - \theta_{7} + \frac{\theta_{9}}{2} \right)} - 0.380688956619433 \cos{\left(\theta_{6} + \theta_{7} - \theta_{8} \right)} - 0.380701120667437 \cos{\left(\theta_{6} + \theta_{7} - \frac{\theta_{8}}{2} \right)} + 0.380676792571429 \cos{\left(\theta_{6} + \theta_{7} + \frac{\theta_{8}}{2} \right)} + 0.380688956619433 \cos{\left(\theta_{6} + \theta_{7} + \theta_{8} \right)} + 0.380676792571429 \cos{\left(\theta_{6} + \theta_{7} - \frac{\theta_{9}}{2} \right)} - 0.380701120667437 \cos{\left(\theta_{6} + \theta_{7} + \frac{\theta_{9}}{2} \right)} - 0.761353585142858 \cos{\left(\theta_{6} + \frac{\theta_{8}}{2} + \frac{\theta_{9}}{2} \right)} + 2.4328096007863 \cdot 10^{-5} \cos{\left(- \theta_{7} + \frac{\theta_{8}}{2} + \frac{\theta_{9}}{2} \right)} + 2.4328096007863 \cdot 10^{-5} \cos{\left(\theta_{7} + \frac{\theta_{8}}{2} + \frac{\theta_{9}}{2} \right)} - 0.380701120667437 \cos{\left(- \theta_{6} + \theta_{7} + \frac{\theta_{8}}{2} + \frac{\theta_{9}}{2} \right)} + 0.380676792571429 \cos{\left(\theta_{6} - \theta_{7} + \frac{\theta_{8}}{2} + \frac{\theta_{9}}{2} \right)} - 0.380701120667437 \cos{\left(\theta_{6} + \theta_{7} - \frac{\theta_{8}}{2} - \frac{\theta_{9}}{2} \right)} + 0.380676792571429 \cos{\left(\theta_{6} + \theta_{7} + \frac{\theta_{8}}{2} + \frac{\theta_{9}}{2} \right)} - 4.86561920156945 \cdot 10^{-5}\right)^{2} + 0.00818328463224794 \left(- 4.86561920156721 \cdot 10^{-5} \sin{\left(\frac{\theta_{8}}{2} \right)} - 4.86561920156721 \cdot 10^{-5} \sin{\left(\frac{\theta_{9}}{2} \right)} - 0.761377913238866 \sin{\left(\theta_{6} - \theta_{8} \right)} - 0.761402241334874 \sin{\left(\theta_{6} - \frac{\theta_{8}}{2} \right)} - 0.761353585142858 \sin{\left(\theta_{6} + \frac{\theta_{8}}{2} \right)} - 0.761377913238866 \sin{\left(\theta_{6} + \theta_{8} \right)} + 0.761402241334874 \sin{\left(\theta_{6} - \frac{\theta_{9}}{2} \right)} + 0.761353585142858 \sin{\left(\theta_{6} + \frac{\theta_{9}}{2} \right)} + 2.43280960078361 \cdot 10^{-5} \sin{\left(\theta_{7} - \frac{\theta_{8}}{2} \right)} - 2.43280960078361 \cdot 10^{-5} \sin{\left(\theta_{7} + \frac{\theta_{8}}{2} \right)} - 2.43280960078361 \cdot 10^{-5} \sin{\left(\theta_{7} - \frac{\theta_{9}}{2} \right)} + 2.43280960078361 \cdot 10^{-5} \sin{\left(\theta_{7} + \frac{\theta_{9}}{2} \right)} - 4.86561920156721 \cdot 10^{-5} \sin{\left(\frac{\theta_{8}}{2} + \frac{\theta_{9}}{2} \right)} + 0.380701120667437 \sin{\left(- \theta_{6} + \theta_{7} + \frac{\theta_{8}}{2} \right)} + 0.380688956619433 \sin{\left(- \theta_{6} + \theta_{7} + \theta_{8} \right)} + 0.380701120667437 \sin{\left(- \theta_{6} + \theta_{7} + \frac{\theta_{9}}{2} \right)} - 0.761353585142858 \sin{\left(- \theta_{6} + \frac{\theta_{8}}{2} + \frac{\theta_{9}}{2} \right)} - 0.380676792571429 \sin{\left(\theta_{6} - \theta_{7} + \frac{\theta_{8}}{2} \right)} - 0.380688956619433 \sin{\left(\theta_{6} - \theta_{7} + \theta_{8} \right)} - 0.380676792571429 \sin{\left(\theta_{6} - \theta_{7} + \frac{\theta_{9}}{2} \right)} - 0.380688956619433 \sin{\left(\theta_{6} + \theta_{7} - \theta_{8} \right)} - 0.380701120667437 \sin{\left(\theta_{6} + \theta_{7} - \frac{\theta_{8}}{2} \right)} - 0.380676792571429 \sin{\left(\theta_{6} + \theta_{7} + \frac{\theta_{8}}{2} \right)} - 0.380688956619433 \sin{\left(\theta_{6} + \theta_{7} + \theta_{8} \right)} - 0.380701120667437 \sin{\left(\theta_{6} + \theta_{7} - \frac{\theta_{9}}{2} \right)} - 0.380676792571429 \sin{\left(\theta_{6} + \theta_{7} + \frac{\theta_{9}}{2} \right)} + 0.761402241334874 \sin{\left(\theta_{6} + \frac{\theta_{8}}{2} + \frac{\theta_{9}}{2} \right)} + 2.43280960078361 \cdot 10^{-5} \sin{\left(- \theta_{7} + \frac{\theta_{8}}{2} + \frac{\theta_{9}}{2} \right)} + 2.43280960078361 \cdot 10^{-5} \sin{\left(\theta_{7} + \frac{\theta_{8}}{2} + \frac{\theta_{9}}{2} \right)} + 0.380676792571429 \sin{\left(- \theta_{6} + \theta_{7} + \frac{\theta_{8}}{2} + \frac{\theta_{9}}{2} \right)} - 0.380701120667437 \sin{\left(\theta_{6} - \theta_{7} + \frac{\theta_{8}}{2} + \frac{\theta_{9}}{2} \right)} - 0.380676792571429 \sin{\left(\theta_{6} + \theta_{7} - \frac{\theta_{8}}{2} - \frac{\theta_{9}}{2} \right)} - 0.380701120667437 \sin{\left(\theta_{6} + \theta_{7} + \frac{\theta_{8}}{2} + \frac{\theta_{9}}{2} \right)} + 0.479249714471874 \cos{\left(\theta_{6} \right)} + 0.479249714471874 \cos{\left(\theta_{7} \right)} - 0.479249714471875 \cos{\left(\frac{\theta_{8}}{2} \right)} - 0.479249714471875 \cos{\left(\frac{\theta_{9}}{2} \right)} + 0.239624857235937 \cos{\left(\theta_{6} - \theta_{7} \right)} + 0.239624857235937 \cos{\left(\theta_{6} + \theta_{7} \right)} + 0.760375142764063 \cos{\left(\theta_{6} - \theta_{8} \right)} + 1.0 \cos{\left(\theta_{6} - \frac{\theta_{8}}{2} \right)} - 0.520750285528125 \cos{\left(\theta_{6} + \frac{\theta_{8}}{2} \right)} - 0.760375142764063 \cos{\left(\theta_{6} + \theta_{8} \right)} + 0.520750285528125 \cos{\left(\theta_{6} - \frac{\theta_{9}}{2} \right)} - 1.0 \cos{\left(\theta_{6} + \frac{\theta_{9}}{2} \right)} - 0.239624857235937 \cos{\left(\theta_{7} - \frac{\theta_{8}}{2} \right)} - 0.239624857235937 \cos{\left(\theta_{7} + \frac{\theta_{8}}{2} \right)} + 0.239624857235937 \cos{\left(\theta_{7} - \frac{\theta_{9}}{2} \right)} + 0.239624857235937 \cos{\left(\theta_{7} + \frac{\theta_{9}}{2} \right)} + 0.479249714471875 \cos{\left(\frac{\theta_{8}}{2} + \frac{\theta_{9}}{2} \right)} + 0.5 \cos{\left(- \theta_{6} + \theta_{7} + \frac{\theta_{8}}{2} \right)} + 0.380187571382031 \cos{\left(- \theta_{6} + \theta_{7} + \theta_{8} \right)} - 0.260375142764063 \cos{\left(- \theta_{6} + \theta_{7} + \frac{\theta_{9}}{2} \right)} - \cos{\left(- \theta_{6} + \frac{\theta_{8}}{2} + \frac{\theta_{9}}{2} \right)} - 0.260375142764063 \cos{\left(\theta_{6} - \theta_{7} + \frac{\theta_{8}}{2} \right)} - 0.380187571382031 \cos{\left(\theta_{6} - \theta_{7} + \theta_{8} \right)} + 0.5 \cos{\left(\theta_{6} - \theta_{7} + \frac{\theta_{9}}{2} \right)} + 0.380187571382031 \cos{\left(\theta_{6} + \theta_{7} - \theta_{8} \right)} + 0.5 \cos{\left(\theta_{6} + \theta_{7} - \frac{\theta_{8}}{2} \right)} - 0.260375142764063 \cos{\left(\theta_{6} + \theta_{7} + \frac{\theta_{8}}{2} \right)} - 0.380187571382031 \cos{\left(\theta_{6} + \theta_{7} + \theta_{8} \right)} - 0.260375142764063 \cos{\left(\theta_{6} + \theta_{7} - \frac{\theta_{9}}{2} \right)} + 0.5 \cos{\left(\theta_{6} + \theta_{7} + \frac{\theta_{9}}{2} \right)} + 0.520750285528125 \cos{\left(\theta_{6} + \frac{\theta_{8}}{2} + \frac{\theta_{9}}{2} \right)} - 0.239624857235937 \cos{\left(- \theta_{7} + \frac{\theta_{8}}{2} + \frac{\theta_{9}}{2} \right)} - 0.239624857235937 \cos{\left(\theta_{7} + \frac{\theta_{8}}{2} + \frac{\theta_{9}}{2} \right)} + 0.5 \cos{\left(- \theta_{6} + \theta_{7} + \frac{\theta_{8}}{2} + \frac{\theta_{9}}{2} \right)} - 0.260375142764063 \cos{\left(\theta_{6} - \theta_{7} + \frac{\theta_{8}}{2} + \frac{\theta_{9}}{2} \right)} + 0.5 \cos{\left(\theta_{6} + \theta_{7} - \frac{\theta_{8}}{2} - \frac{\theta_{9}}{2} \right)} - 0.260375142764063 \cos{\left(\theta_{6} + \theta_{7} + \frac{\theta_{8}}{2} + \frac{\theta_{9}}{2} \right)} + 0.479249714471874\right)^{2} + 0.00930674450091877 \left(0.846212986949131 \sin{\left(\frac{\theta_{8}}{2} \right)} + 0.846212986949131 \sin{\left(\frac{\theta_{9}}{2} \right)} - 0.29786835563659 \sin{\left(\theta_{6} - \theta_{8} \right)} + 0.125238137837975 \sin{\left(\theta_{6} - \frac{\theta_{8}}{2} \right)} - 0.720974849111156 \sin{\left(\theta_{6} + \frac{\theta_{8}}{2} \right)} - 0.29786835563659 \sin{\left(\theta_{6} + \theta_{8} \right)} - 0.125238137837975 \sin{\left(\theta_{6} - \frac{\theta_{9}}{2} \right)} + 0.720974849111156 \sin{\left(\theta_{6} + \frac{\theta_{9}}{2} \right)} - 0.423106493474566 \sin{\left(\theta_{7} - \frac{\theta_{8}}{2} \right)} + 0.423106493474566 \sin{\left(\theta_{7} + \frac{\theta_{8}}{2} \right)} + 0.423106493474566 \sin{\left(\theta_{7} - \frac{\theta_{9}}{2} \right)} - 0.423106493474566 \sin{\left(\theta_{7} + \frac{\theta_{9}}{2} \right)} + 0.846212986949131 \sin{\left(\frac{\theta_{8}}{2} + \frac{\theta_{9}}{2} \right)} - 0.0626190689189875 \sin{\left(- \theta_{6} + \theta_{7} + \frac{\theta_{8}}{2} \right)} + 0.148934177818295 \sin{\left(- \theta_{6} + \theta_{7} + \theta_{8} \right)} - 0.0626190689189875 \sin{\left(- \theta_{6} + \theta_{7} + \frac{\theta_{9}}{2} \right)} - 0.720974849111156 \sin{\left(- \theta_{6} + \frac{\theta_{8}}{2} + \frac{\theta_{9}}{2} \right)} - 0.360487424555578 \sin{\left(\theta_{6} - \theta_{7} + \frac{\theta_{8}}{2} \right)} - 0.148934177818295 \sin{\left(\theta_{6} - \theta_{7} + \theta_{8} \right)} - 0.360487424555578 \sin{\left(\theta_{6} - \theta_{7} + \frac{\theta_{9}}{2} \right)} - 0.148934177818295 \sin{\left(\theta_{6} + \theta_{7} - \theta_{8} \right)} + 0.0626190689189875 \sin{\left(\theta_{6} + \theta_{7} - \frac{\theta_{8}}{2} \right)} - 0.360487424555578 \sin{\left(\theta_{6} + \theta_{7} + \frac{\theta_{8}}{2} \right)} - 0.148934177818295 \sin{\left(\theta_{6} + \theta_{7} + \theta_{8} \right)} + 0.0626190689189875 \sin{\left(\theta_{6} + \theta_{7} - \frac{\theta_{9}}{2} \right)} - 0.360487424555578 \sin{\left(\theta_{6} + \theta_{7} + \frac{\theta_{9}}{2} \right)} - 0.125238137837975 \sin{\left(\theta_{6} + \frac{\theta_{8}}{2} + \frac{\theta_{9}}{2} \right)} - 0.423106493474566 \sin{\left(- \theta_{7} + \frac{\theta_{8}}{2} + \frac{\theta_{9}}{2} \right)} - 0.423106493474566 \sin{\left(\theta_{7} + \frac{\theta_{8}}{2} + \frac{\theta_{9}}{2} \right)} + 0.360487424555578 \sin{\left(- \theta_{6} + \theta_{7} + \frac{\theta_{8}}{2} + \frac{\theta_{9}}{2} \right)} + 0.0626190689189875 \sin{\left(\theta_{6} - \theta_{7} + \frac{\theta_{8}}{2} + \frac{\theta_{9}}{2} \right)} - 0.360487424555578 \sin{\left(\theta_{6} + \theta_{7} - \frac{\theta_{8}}{2} - \frac{\theta_{9}}{2} \right)} + 0.0626190689189875 \sin{\left(\theta_{6} + \theta_{7} + \frac{\theta_{8}}{2} + \frac{\theta_{9}}{2} \right)} - \cos{\left(\theta_{6} \right)} - \cos{\left(\theta_{7} \right)} + \cos{\left(\frac{\theta_{8}}{2} \right)} + \cos{\left(\frac{\theta_{9}}{2} \right)} - 0.5 \cos{\left(\theta_{6} - \theta_{7} \right)} - 0.5 \cos{\left(\theta_{6} + \theta_{7} \right)} - 0.00269431212566768 \cos{\left(\theta_{6} - \theta_{8} \right)} - 0.502694312125668 \cos{\left(\theta_{6} - \frac{\theta_{8}}{2} \right)} - 0.497305687874332 \cos{\left(\theta_{6} + \frac{\theta_{8}}{2} \right)} + 0.00269431212566768 \cos{\left(\theta_{6} + \theta_{8} \right)} + 0.497305687874332 \cos{\left(\theta_{6} - \frac{\theta_{9}}{2} \right)} + 0.502694312125668 \cos{\left(\theta_{6} + \frac{\theta_{9}}{2} \right)} + 0.5 \cos{\left(\theta_{7} - \frac{\theta_{8}}{2} \right)} + 0.5 \cos{\left(\theta_{7} + \frac{\theta_{8}}{2} \right)} - 0.5 \cos{\left(\theta_{7} - \frac{\theta_{9}}{2} \right)} - 0.5 \cos{\left(\theta_{7} + \frac{\theta_{9}}{2} \right)} - \cos{\left(\frac{\theta_{8}}{2} + \frac{\theta_{9}}{2} \right)} - 0.251347156062834 \cos{\left(- \theta_{6} + \theta_{7} + \frac{\theta_{8}}{2} \right)} - 0.00134715606283384 \cos{\left(- \theta_{6} + \theta_{7} + \theta_{8} \right)} - 0.248652843937166 \cos{\left(- \theta_{6} + \theta_{7} + \frac{\theta_{9}}{2} \right)} + 0.502694312125668 \cos{\left(- \theta_{6} + \frac{\theta_{8}}{2} + \frac{\theta_{9}}{2} \right)} - 0.248652843937166 \cos{\left(\theta_{6} - \theta_{7} + \frac{\theta_{8}}{2} \right)} + 0.00134715606283384 \cos{\left(\theta_{6} - \theta_{7} + \theta_{8} \right)} - 0.251347156062834 \cos{\left(\theta_{6} - \theta_{7} + \frac{\theta_{9}}{2} \right)} - 0.00134715606283384 \cos{\left(\theta_{6} + \theta_{7} - \theta_{8} \right)} - 0.251347156062834 \cos{\left(\theta_{6} + \theta_{7} - \frac{\theta_{8}}{2} \right)} - 0.248652843937166 \cos{\left(\theta_{6} + \theta_{7} + \frac{\theta_{8}}{2} \right)} + 0.00134715606283384 \cos{\left(\theta_{6} + \theta_{7} + \theta_{8} \right)} - 0.248652843937166 \cos{\left(\theta_{6} + \theta_{7} - \frac{\theta_{9}}{2} \right)} - 0.251347156062834 \cos{\left(\theta_{6} + \theta_{7} + \frac{\theta_{9}}{2} \right)} + 0.497305687874332 \cos{\left(\theta_{6} + \frac{\theta_{8}}{2} + \frac{\theta_{9}}{2} \right)} + 0.5 \cos{\left(- \theta_{7} + \frac{\theta_{8}}{2} + \frac{\theta_{9}}{2} \right)} + 0.5 \cos{\left(\theta_{7} + \frac{\theta_{8}}{2} + \frac{\theta_{9}}{2} \right)} - 0.251347156062834 \cos{\left(- \theta_{6} + \theta_{7} + \frac{\theta_{8}}{2} + \frac{\theta_{9}}{2} \right)} - 0.248652843937166 \cos{\left(\theta_{6} - \theta_{7} + \frac{\theta_{8}}{2} + \frac{\theta_{9}}{2} \right)} - 0.251347156062834 \cos{\left(\theta_{6} + \theta_{7} - \frac{\theta_{8}}{2} - \frac{\theta_{9}}{2} \right)} - 0.248652843937166 \cos{\left(\theta_{6} + \theta_{7} + \frac{\theta_{8}}{2} + \frac{\theta_{9}}{2} \right)} - 1\right)^{2}}"

In [107]:
latex = latex.replace("\left", "").replace("\right", "")

latex = latex.replace("\f", "f")

latex = latex.replace("\theta_", "t").replace("\sin", "sin").replace("\cos", "cos")

for i in range(10):
    latex = latex.replace("t{"+str(i)+"}", "t"+str(i))

latex = latex.replace("}{", "}/{").replace("frac", "").replace("{(", "{").replace(")}", "}").replace("{", "(").replace("}", ")").replace(" ", "")

latex = latex.replace("t6", "x").replace("t7", "x").replace("t8", "y").replace("t9", "y")

latex = latex.replace("(2)", "2") #.replace("(x)", "x").replace("(y)", "y")

In [110]:
latex

'(0.418614155505566)/(0.00930674450091877(-sin((y)/2)-sin((y)/2)-0.00269431212566771sin(x-y)-0.502694312125668sin(x-(y)/2)+0.497305687874332sin(x+(y)/2)-0.00269431212566771sin(x+y)+0.502694312125668sin(x-(y)/2)-0.497305687874332sin(x+(y)/2)+0.5sin(x-(y)/2)-0.5sin(x+(y)/2)-0.5sin(x-(y)/2)+0.5sin(x+(y)/2)-sin((y)/2+(y)/2)+0.251347156062834sin(-x+x+(y)/2)+0.00134715606283385sin(-x+x+y)+0.251347156062834sin(-x+x+(y)/2)+0.497305687874332sin(-x+(y)/2+(y)/2)+0.248652843937166sin(x-x+(y)/2)-0.00134715606283385sin(x-x+y)+0.248652843937166sin(x-x+(y)/2)-0.00134715606283385sin(x+x-y)-0.251347156062834sin(x+x-(y)/2)+0.248652843937166sin(x+x+(y)/2)-0.00134715606283385sin(x+x+y)-0.251347156062834sin(x+x-(y)/2)+0.248652843937166sin(x+x+(y)/2)+0.502694312125668sin(x+(y)/2+(y)/2)+0.5sin(-x+(y)/2+(y)/2)+0.5sin(x+(y)/2+(y)/2)-0.248652843937166sin(-x+x+(y)/2+(y)/2)-0.251347156062834sin(x-x+(y)/2+(y)/2)+0.248652843937166sin(x+x-(y)/2-(y)/2)-0.251347156062834sin(x+x+(y)/2+(y)/2)-0.846212986949131cos(x)-0.84

In [25]:
(L(p0000), L(p0_denom))

(773, 1675)

In [7]:
# derive expression for <0000|psi10>

raw0000 = braket(idx_to_vect("0000"), psi14).expand(trig=True).expand()

braket0000 = simplify(raw0000, maxn=17, chop=True)
temp = braket0000
for a in preorder_traversal(temp):
    if isinstance(a, Float):
        braket0000 = braket0000.subs(a, round(a, 15))

display(braket0000)

In [ ]:
# calculate P(|0000>) = |<0000|psi10>|

p0000 = simplify(re(braket0000)*re(braket0000)+im(braket0000)*im(braket0000))

display(p0000)

In [ ]:
# derive expression for <0100|psi10>

raw0100 = braket(idx_to_vect("0100"), psi14).expand(trig=True).expand()

braket0100 = simplify(raw0100, maxn=17, chop=True)
temp = braket0100
for a in preorder_traversal(temp):
    if isinstance(a, Float):
        braket0100 = braket0100.subs(a, round(a, 15))

display(braket0100)

In [ ]:
# calculate P(|0100>) = |<0100|psi10>|

p0100 = simplify(re(braket0100)*re(braket0100)+im(braket0100)*im(braket0100))

display(p0100)

In [ ]:
p0_denom = simplify(p0000+p0100)

display(p0_denom)

In [ ]:
# calculate P(O=|0>) = P(|0000>)/( P(|0000>)+P(|0100>) )

p0 = simplify(p0000/p0_denom)

display(p0)

In [ ]:
tau = 2*3.141592653589796

verb_0 = tau*0
verb_1 = tau*0
verb_2 = tau*0
verb_3 = tau*0
verb_4 = tau*0
verb_5 = tau*0
adj_0 = tau*0
adj_1 = tau*0
adj_2 = tau*0
adj_3 = tau*0

In [ ]:
p0.subs([(t0, verb_0), (t1, verb_1), (t2, verb_2), (t3, verb_3), (t4, verb_4), (t5, verb_5), (t6, adj_0), (t7, adj_1), (t8, adj_2), (t9, adj_3)])

## SXA

In [8]:
t0 = symbols('\\theta_0', real=True)
t1 = symbols('\\theta_1', real=True)
t2 = symbols('\\theta_2', real=True)
t3 = symbols('\\theta_3', real=True)
pi = symbols('\\pi', real=True)

In [9]:
phi0 = idx_to_vect("00")

phi1 = tfv(mtp2([rx(t0), rx(t1)]), phi0)

phi2 = tfv(cgate2(rz(t2), 0, 1), phi1)

phi3 = tfv(cgate2(rz(t3), 1, 0), phi2)

phi4 = tfv(mtp2([rx(t0), rx(t1)]), phi3)

phi5 = tfv(cgate2(rz(t2), 0, 1), phi4)

phi6 = tfv(cgate2(rz(t3), 1, 0), phi5)

In [10]:
# derive expression for <00|phi6>

raw00 = braket(idx_to_vect("00"), phi6).expand(trig=True).expand()

braket00 = simplify(raw00, maxn=17, chop=True)
temp = braket00
for a in preorder_traversal(temp):
    if isinstance(a, Float):
        braket00 = braket00.subs(a, round(a, 15))

display(braket00)

 ⅈ⋅\theta₂   ⅈ⋅\theta₃                                                        
 ───────── + ─────────                                                        
     2           2        2⎛\theta₀⎞    2⎛\theta₁⎞      2⎛\theta₀⎞    2⎛\theta
ℯ                     ⋅sin ⎜───────⎟⋅sin ⎜───────⎟ + cos ⎜───────⎟⋅cos ⎜──────
                           ⎝   2   ⎠     ⎝   2   ⎠       ⎝   2   ⎠     ⎝   2  

      -ⅈ⋅\theta₃                                 -ⅈ⋅\theta₂                   
      ───────────                                ───────────                  
₁⎞         2         2⎛\theta₁⎞    2⎛\theta₀⎞         2         2⎛\theta₀⎞    
─⎟ - ℯ           ⋅sin ⎜───────⎟⋅cos ⎜───────⎟ - ℯ           ⋅sin ⎜───────⎟⋅cos
 ⎠                    ⎝   2   ⎠     ⎝   2   ⎠                    ⎝   2   ⎠    

          
          
2⎛\theta₁⎞
 ⎜───────⎟
 ⎝   2   ⎠

In [11]:
# calculate P(|00>) = |<00|phi6>|

p00 = simplify(re(braket00)*re(braket00)+im(braket00)*im(braket00))

display(p00)

                                                                              
⎛   2⎛\theta₀⎞    2⎛\theta₁⎞    ⎛\theta₂   \theta₃⎞      2⎛\theta₀⎞    ⎛\theta
⎜sin ⎜───────⎟⋅sin ⎜───────⎟⋅sin⎜─────── + ───────⎟ + sin ⎜───────⎟⋅sin⎜──────
⎝    ⎝   2   ⎠     ⎝   2   ⎠    ⎝   2         2   ⎠       ⎝   2   ⎠    ⎝   2  

                                                            2                 
₂⎞    2⎛\theta₁⎞      2⎛\theta₁⎞    ⎛\theta₃⎞    2⎛\theta₀⎞⎞    ⎛   2⎛\theta₀⎞
─⎟⋅cos ⎜───────⎟ + sin ⎜───────⎟⋅sin⎜───────⎟⋅cos ⎜───────⎟⎟  + ⎜sin ⎜───────⎟
 ⎠     ⎝   2   ⎠       ⎝   2   ⎠    ⎝   2   ⎠     ⎝   2   ⎠⎠    ⎝    ⎝   2   ⎠

                                                                              
    2⎛\theta₁⎞    ⎛\theta₂   \theta₃⎞      2⎛\theta₀⎞    2⎛\theta₁⎞    ⎛\theta
⋅sin ⎜───────⎟⋅cos⎜─────── + ───────⎟ - sin ⎜───────⎟⋅cos ⎜───────⎟⋅cos⎜──────
     ⎝   2   ⎠    ⎝   2         2   ⎠       ⎝   2   ⎠     ⎝   2   ⎠    ⎝   2  

                                                 

In [12]:
# derive expression for <01|psi10>

raw01 = braket(idx_to_vect("01"), phi6).expand(trig=True).expand()

braket01 = simplify(raw01, maxn=17, chop=True)
temp = braket01
for a in preorder_traversal(temp):
    if isinstance(a, Float):
        braket01 = braket01.subs(a, round(a, 15))

display(braket01)

  ⎛ 3⋅ⅈ⋅\theta₃                  ⅈ⋅(\theta₂ + 2⋅\theta₃)                      
  ⎜ ───────────                  ───────────────────────                      
  ⎜      2         2⎛\theta₀⎞               2               2⎛\theta₀⎞    ⅈ⋅(\
ⅈ⋅⎜ℯ           ⋅sin ⎜───────⎟ - ℯ                       ⋅cos ⎜───────⎟ + ℯ    
  ⎝                 ⎝   2   ⎠                                ⎝   2   ⎠        

                                     ⅈ⋅(\theta₂ + 3⋅\theta₃)              ⎞  -
                                     ───────────────────────              ⎟  ─
theta₂ + 2⋅\theta₃)    2⎛\theta₀⎞               2               2⎛\theta₀⎞⎟   
                   ⋅sin ⎜───────⎟ - ℯ                       ⋅cos ⎜───────⎟⎟⋅ℯ 
                        ⎝   2   ⎠                                ⎝   2   ⎠⎠   

ⅈ⋅(\theta₂ + 4⋅\theta₃)                           
────────────────────────                          
           2                ⎛\theta₁⎞    ⎛\theta₁⎞
                        ⋅sin⎜───────⎟⋅cos⎜───────⎟
    

In [13]:
# calculate P(|01>) = |<01|phi6>|

p01 = simplify(re(braket01)*re(braket01)+im(braket01)*im(braket01))

display(p01)

⎛                                               2             ⎛\theta₂⎞      2
⎜                                            sin (\theta₀)⋅cos⎜───────⎟   sin 
⎜     2             ⎛\theta₂⎞    ⎛\theta₃⎞                    ⎝   2   ⎠       
⎜- sin (\theta₀)⋅cos⎜───────⎟⋅cos⎜───────⎟ - ────────────────────────── - ────
⎝                   ⎝   2   ⎠    ⎝   2   ⎠               2                    

             ⎛\theta₃⎞      2             ⎛\theta₂          ⎞      2          
(\theta₀)⋅cos⎜───────⎟   sin (\theta₀)⋅cos⎜─────── + \theta₃⎟   sin (\theta₀)⋅
             ⎝   2   ⎠                    ⎝   2             ⎠                 
────────────────────── - ──────────────────────────────────── - ──────────────
        2                                 2                                   

   ⎛          \theta₃⎞                                                        
cos⎜\theta₂ + ───────⎟                                                        
   ⎝             2   ⎠      2                     

In [14]:
p0_ = simplify(p00+p01)

display(p0_)

                                                                              
                                                                              
⎛   2⎛\theta₀⎞    2⎛\theta₁⎞    ⎛\theta₂   \theta₃⎞      2⎛\theta₀⎞    ⎛\theta
⎜sin ⎜───────⎟⋅sin ⎜───────⎟⋅sin⎜─────── + ───────⎟ + sin ⎜───────⎟⋅sin⎜──────
⎝    ⎝   2   ⎠     ⎝   2   ⎠    ⎝   2         2   ⎠       ⎝   2   ⎠    ⎝   2  

                                                                              
                                                            2                 
₂⎞    2⎛\theta₁⎞      2⎛\theta₁⎞    ⎛\theta₃⎞    2⎛\theta₀⎞⎞    ⎛   2⎛\theta₀⎞
─⎟⋅cos ⎜───────⎟ + sin ⎜───────⎟⋅sin⎜───────⎟⋅cos ⎜───────⎟⎟  + ⎜sin ⎜───────⎟
 ⎠     ⎝   2   ⎠       ⎝   2   ⎠    ⎝   2   ⎠     ⎝   2   ⎠⎠    ⎝    ⎝   2   ⎠

                                                                              
                                                                              
    2⎛\theta₁⎞    ⎛\theta₂   \theta₃⎞      2⎛\thet

In [15]:
# calculate P(O=|0>) = P(|00>)/( P(|00>)+P(|01>) )

f0 = simplify(p00/p0_)

display(f0)

                                                                              
                                                                              
                                                                              
                                                                              
──────────────────────────────────────────────────────────────────────────────
                                                                              
  ⎛   2⎛\theta₀⎞    2⎛\theta₁⎞    ⎛\theta₂   \theta₃⎞      2⎛\theta₀⎞    ⎛\the
2⋅⎜sin ⎜───────⎟⋅sin ⎜───────⎟⋅sin⎜─────── + ───────⎟ + sin ⎜───────⎟⋅sin⎜────
  ⎝    ⎝   2   ⎠     ⎝   2   ⎠    ⎝   2         2   ⎠       ⎝   2   ⎠    ⎝   2

                                                                              
                                                                              
                                                                              
                                                   

In [ ]:
tau = 2*3.141592653589793

adj_0 = tau*0
adj_1 = tau*0
adj_2 = tau*0
adj_3 = tau*0

In [ ]:
f0.subs([(t0, adj_0), (t1, adj_1), (t2, adj_2), (t3, adj_3)])